# Importing dataset and setting up

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()








# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id') 
X_test = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

# Missing values

In [ ]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

####################################################################################

# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])




####################################################################################
# Option 1
# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))
####################################################################################




####################################################################################
# Option 2
# Imputaion by 'mean' (can change it see docs)
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))
####################################################################################





####################################################################################
# Option 3
# Imputation with creation of extra col Ture/Flase whether impuated or not
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns # can actually just put this in the  .fit_traonsform() etc
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))



# Catagorical variables

In [ ]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)


####################################################################################
# Option 1
# Drop catagorical values
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))
####################################################################################



####################################################################################
# Option 2
# Ordinal Encoding
from sklearn.preprocessing import OrdinalEncoder



# This is needed to ensure the catagories in the valid and training set line up
# Categorical columns in the training data
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_valid[col]).issubset(set(X_train[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder 
ordinal_encoder = OrdinalEncoder() 
label_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
label_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])





print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))
####################################################################################


####################################################################################
# Option 3
# One Hot encoding
from sklearn.preprocessing import OneHotEncoder

# find out which cols have few number of cats (low cardinality)
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)



# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))
####################################################################################


####################################################################################



####################################################################################





# Testing different models

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Define the models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

# Function for comparing different models
def score_model(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)


for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

# To score one model
# def score_dataset(X_train, X_valid, y_train, y_valid):
#     model = RandomForestRegressor(n_estimators=10, random_state=0)
#     model.fit(X_train, y_train)
#     preds = model.predict(X_valid)
#     return mean_absolute_error(y_valid, preds)


my_model = best_model

# Fit the model to the training data
my_model.fit(X, y)

# Generate test predictions
preds_test = my_model.predict(X_test)

# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

# Pipelines

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])



from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

# Cross Validation

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])


# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

print("Average MAE score (across experiments):")
print(scores.mean())

In [ ]:
def get_score(n_estimators):
    """Return the average MAE over 3 CV folds of random forest model.
    
    Keyword argument:
    n_estimators -- the number of trees in the forest
    """
    my_pipeline = Pipeline(steps=[
        ('preprocessor', SimpleImputer()),
        ('model', RandomForestRegressor(n_estimators, random_state=0))
    ])
    
    scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=3,
                              scoring='neg_mean_absolute_error')
    return scores.mean()

results = {i: get_score(i) for i in range(50,450,50)}


# XGBoost

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Define the model
my_model_2 = XGBRegressor(random_state = 0, 
                          n_estimators = 1000, 
                          learning_rate = 0.05,
                          early_stopping_rounds=5) # Your code here

# Fit the model
my_model_2.fit(X_train, y_train,  
                eval_set=[(X_valid, y_valid)], 
                verbose=False)

# Get predictions
predictions_2 = my_model_2.predict(X_valid) # Your code here

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)



In [96]:
import collections
arr = collections.Counter('AAAAB')
print(arr.keys())

dict_keys(['A', 'B'])


In [87]:
import collections
class Solution_3:
    def characterReplacement(self, s, k):
        maxlen, largestCount = 0, 0
        arr = collections.Counter()
        for idx in range(len(s)):
            arr[s[idx]] += 1
            largestCount = max(largestCount, arr[s[idx]])
            if maxlen - largestCount >= k:
                arr[s[idx - maxlen]] -= 1
            else:
                maxlen += 1
        return maxlen

import collections
class Solution_2:
    def characterReplacement(self, s, k):
        cs = set(s)
        maxlen = 0
        for c in cs:
            largestCount = 0
            current_len = 0
            for idx in range(len(s)):
                if c == s[idx]:
                    largestCount += 1
                if current_len - largestCount >= k:
                    if s[idx - current_len] == c:
                        largestCount -= 1
                else:
                    current_len += 1
            maxlen = max(current_len, maxlen)
        return maxlen

class Solution:
    def characterReplacement(self, s: str, k: int) -> int:
        self.s = s
        self.k = k

        lo = 1
        hi = len(s)
        result = 1

        # Binary search
        while lo <= hi:
            mid = (lo + hi) // 2 
            if self.isValidString(mid):
                result = max(result,mid)
                lo = mid + 1
            else:
                hi = mid - 1

        return result


    def isValidString(self, length):
        l = 0
        boo = False

        # Buidling freduqency dictionary
        freq_dict = {}
        for i in range(length):
            if self.s[i] in freq_dict:
                freq_dict[self.s[i]] += 1
            else:
                freq_dict[self.s[i]] = 1
        
        for r in range((length-1), len(self.s)):

            # Check if this window is true
            max_freq = max(freq_dict.values())
            n_other = length - max_freq
            if n_other <= self.k:
                return True


            if r != (len(self.s)-1):
                # Shift window along
                freq_dict[self.s[l]] -= 1
                if self.s[r+1] in freq_dict:
                    freq_dict[self.s[r+1]] += 1
                else:
                    freq_dict[self.s[r+1]] = 1
            l += 1

        return boo
                

obj = Solution()
obj_2 = Solution_2()
obj_3 = Solution_3()

s_arr = [["XXVKJPAAEEVWLJWLTJQSNJUWYSGMUUVXCSSSSWMRSAYDSKBOZVFKYQVUGSKDJRHKKNOSEXBSAJLRQOSDSIKQUXHRRKPFJAYQBAVJOKUZUCWRXOSHSYSXHNULFNSHAKHHMZTDYWKYKWYFYVLQJPHAAWOEVTKWRHDAERIDQFQCEBHIQINNWWJENNXRBQZYXMANEFBHENGOZTRBCQQPHYDVDTXMELWSCJFVIJWRKTKMPUATCSKDLBUGKTYBUNAILJYIFQXJUSFKIBOYKXHTNVCIUJQITZRAVUTIAACMLRZLEDTHCMNFAMVAPQVDXHNBFQTFQGIKYVXVALTZMEHKWEVLKFPEZHJTRDSJYVXLNPWYYCSANQWCGFIFDMTMZBOYBPDAMNIPVRFWFNDCIFDVDDKUQYTGPNLBIDQWNJNCDVGYYYRBWOARXDVNEXIEUKYMJLCJBCPBLVJWYFXGLEVAASDWKPEDKCGKQSYXQTJNSWKUILFQSIPWKIUXVWSHDHXYOSBDHUVDCHDGCCOXQBJXVAWCACTAISCKZFNOMHWCVACIXIUHXIIIROGNDZPXHYTNENYPWNJIDKTNISQOHFAVRNQPGLAOXDBZJZDJIEUBOHWTNEKQKYLZUTIUNXJLLBKSPUUVDNHSSJCFUIXVJTSM",759,],["LVRRTIUXRNGYJHVJNHVXNVQSVZBMSYUHHISETDCWFNGKZNYFRVVGIFXTDXJRFGJIDXESYWVLRXNINFAADBJRYFAZDRYNIZXCGVDZTGTNQYSKRBIZNSPSTJWNTCUZGWGYMDPXKLBOLFCLQBWPYWUFXLVBNXJAYOWRFSQQCWEVMWJYZOVBZTXQDPUMJGEYFATGVVUIOJIINOTAJIURHOHKCTSEKNDWZIZRIJSPYBSZIHXVOTYCRZETVEZALFCXCQNWRPVCKETSKGWYMBZPFVAKVAHJSPIOWZZTFIMTRPZVUXCUMQFJGMIXGGGOOUXLWFZYIZCBUQXGLFSVJSTTDPCAJKWIEULMWNKIRLFHZRCPNQXPOPCGGUPCINJFSGLJRFASYCVPDZMNEGWDKFJKAGNUWSGRPYQPFDXKBFTKJHZTIDXQYMDSIIRRCDUHCQCO",301,],["OFFOKTDOZGDBBPYDROIUKXPZOR",397,],["BBBASVOBBWARUPRDKEVEWHKXVKJQVTIZLFRMQEWFRTWLZZZSXSWWNZIICEGWYOBTQOEZKJZKGXKSZPKOWEUBARGRBYHVWRXVICIDPPAFBJPVYYQRIPWEKQLRYXMKLWGJENJAGZPAUJHTALGRLMFSRXWJWIUFLFDOCHBHJBEANCSODZJSFCNDGEOFBVQWFXERQBGIDFNDZTVOWXEWXLOTMQRLSMQQWBFPWFAVXCGAKIKECMCSGIWWOXPJGLQNNTDGGFYOYATAWNMFYTHAWSNOSROTYGPCWFYBRPPVLYYNRKMBPLFJSPAXUVEICCNOTNWNMEVLWSSQOOCYWRRWNQASWYLRKAQSTBFOTTYZFRQQFGWSIGVAKAHRQGRDEHOWKNSJUZKFVXUDKDSKHXSWLVOLDVERBQWFHMEJWAFOLEUBFZREKGHXCHHWKCQSQEPLWIZJZRFHVZAHVLVCSIYOIQTMWOXXCZ",832,],["HBYOJZZCFWWCETQBMZYCKAMYYCZMJPMTZILLRDWHAMMATZFPGBVYROTWRINMDGLIBGSHTOVVOCESMUHHRGZVKWUNSROCFPRZXYLEUZENMLNFUJFKWHGODZBIHQHYGUQHZBBVLXAQZUFQGSWEULYXZGGXAEJIHUCUCTBYIWABUQHLHRKHMNXACMYGYKTIZZPDROUOEMBGHBLKEWJZMVSLFVCVVRBUYDPLFKRMHXDQMNBLDRFAQLTIQRQMGIEXDWGTIMVTWQWPBWPSPPDOMOWBHKBQZZFCCKCCEWROIEDBKISPIKCVPQGBMZLPIXIXFCRXZSEXGFTMOIWTDIWCYSQMMMXOCIFNMEJYKJOPORSQFYDFZFPEOBTNGENIBUWHJOCHWDVSZWTXYEDLZCBEFAMMVBCQYTISLDXVZSTDGKYDTZMEZNXJLRWXTKRRIYGLUXSKHVNUXROCLXPTAXQMJKJKGQTCJEUQPCYMBKGFLPDWEBBLFAAUQWTQNYGVBJENLLTGMGOJSVSMNMRTKXJSYZLUQOCCTSCZYFYBDCQLWDFNDJJOPEXLRNSQPMELQHXEPBTFIMJRILZEXZDCFSFNXYDXECPMZQNFIKZSXAVYKKSOMVMNUEVWOBUGUQEMAZYWXXVGWEEAKWQGZVIKIHXFQCRHSIAAXCVDDBNMLTEXBWDYGDUKRGYKTTBOZFKMKDACOZBDXRJXLOHHAHAYCBDQMWUHIFN",987,],["XTIFVYGKWFMTLWWDWNTTPVDSGFPHIQOZRWZFCQNOSHLKNGXDHZNBUKDPWTEMBLRHIGLCMCWPBJTKVSRRWYMJIBHXSQUCGLFIGNQLPZQOVJBPYGHYLMUZHTSCJHFDUGRTNJGJJFDDPRXAGJIVXGCWGQBHBUQAVNVYWIPIQZUNYADUFJDPKAWZHLOIEOHZBLDL",629,],["RQKVCTGVVCWIJKMQOXWHWLXTOQAAHKMKITCPVJJUFXTYIRMUYGIHPHQJXGVGDRVSJWITDMQRYEQEXXDVQFTHYMHJHQGJWGDFYIIWMYEPVWRJGFLXYDJHPWKXLBMIWLYNVUWLQJPSNXBVXNAUJIKVXZBEYIOLXRRLDESAJLTRXQSIJAKNDGRPGIIYUXYTOWMMGMXPRILVVCRVDNEFZIPGUBMAGNBFOVCSVONJRWAUOXRPHDGLBQJYZVJVFMLNLDNGIBQOYMDIROFTETUMBHKVGPLLAGGEKEIHKPYVLWNVDORYKTWLSSVIWXLHTLLCIMRTQJGSTAICDTEPKNWOMOJYIFROLLYDZJJRCLMZWGTALFDHSBYUBTDIJDHDBWFQTCTGXEWHYNCDGTVSRVPSCUWZGARIMTTSRAHPTKTQEYXMITABFGGDKEJRBJRZLXLWTQJETFLHRGIIJFPGJAIHUREOBITJVPNEMIOVJS",859,],["DQQRUQLCMSBOCGQCUCKKTIKJJNQSUZ",224,],["GJBCXYXPUTOPHDTNUUDWRSQBBZJFIUXVYRFWWHSCITJKUZFIQEXPPMRHNJOGZTWTXAARLTPTEXJEUNHQHOBRIMYXVYVLGMHRKGOTNQTENGZCPRHPODMKLAGITSUKSROWCYPIAMIXUBLKVQQRKTBABPMBCBUEZMHWCWKIDHTKIICPPKUNUWJGCZVVTZRBHMWBMRRVMLUPTNSATSRCNCDZSLXBFCALFDMBHPIZWUBACHSCUXMYMGFNPWQQFXXACWDFWCWDIMWLAVEPHEXICCIXCRELOZGXHWWDRQEBOAKEZMPTGYUJVJMRQCIMEEIVRLVTTQDRUDSBNXVBYBPVHHDQFOCMFKDPOUNLPHRSYDIGCBXJEBNBXABGRRGFRFFRDWEALQODZGPKDOINWAULCKXMNMFKSQWSHLORBTQDARVAGNATUGGHUYYBFUVNWTVAMWOTZNRZEULTQOPOVCSWWSKPEZNFDBJEMZPPUHJKRUVOXKFRPXRMVJZAEFWUARFSYRKZEALHACIVRWWLZVZWRXQLAHHPHRKNROBQLDEQQHPLBQFHBSKYKROEPFIYBXFYHRAQUPRFOTJKCKXXKQBBNOVRTFWCAYJAQYKBNNYALVL",264,],["KKGKXUZBRRBZJUHIUYIWLSBGBLWGWNGZOTWEBXLJRSROIJLJGKANMGKIZOKDQRKHELTRHRKZGVIULRNFZSSQMMJMKRKQQROLZRLELSTGZWBYHCDVMLILYKZYLISEFOLRIUBRJQVXERJLSTGGKRQKYRDBPYCBDQDHNKLCHFHVBTHDHKARF",832,],["XDWRRKKKUXBWAIIUWZWUAWOGVYCVELBLJTEUGHIMHQCDURBXIOKAFZNXYKQXIFCRMBTNUQTOVJGHJGBBRDPMINZFSNGGSFSDQAKBBNBGVYDBGGGUNVDGPVSKKJSZGK",301,],["DHGDEGELLZDZAXSKSMZUTNJDRRZRGRJDEMZLKLIBDKIJLMEGPZZIBEHMDSRZSDHRXTNBCMDTCIQZZIDLNWMFDHNIFVJYJDCSULIJOHBGCKDCMVXJJLFRJHHXGZJIGKTRGHENRICNSMVCRQJOLNODXSMEXBKBDAQAGMLQFTFVMVBSXIMIOIBWCCDTTSYBQKZEZAQGHGENRAEBYVIJWCHHYYXIZMSTDHYLWKLTIGEILRVDJRGKVAHLNIVIPMNTEXFNGFJHHLWYZEUCKXFXBKEMVSHWOHUMKVDPIJAMKBEDZVVHQLHGYVXRNXPHOBZPQIKNCIQXVBSAMNODGOKUKIBQUUYUZTCICDMBZVPDJADTVUECOHTPYADKPVMFYWLPJYLUJKUXGJPGLEFCZINBSDLSIAHWUURASJWRSTBDURKFIFCZZKJIJJBIQCKLWILKCKFLLHOMLIPNYKVIWMANZJIGURJQQJBWNRNMKWBLAKTZFBMWDIDNSDGPZUGFZWGUVDFYVZWJUGCOJBTVCJUJWKISEVLAJJOSIULJASBJTKENTHJQQPSDDVRSRVGGODCARUYBXILMRRISLIJIMYKKKCZYWMGPCPWSCYPRAWRHISTZWXBGLIRBKOUOJWHNDAQVYEWJMOLAVIZVWXFFPDVFQXVXSDHLCWXIDFPIPDBGDMEMDHDGSDDMAXSFEUSFEYPIGAWKTKSOUPZUDDGQJVULQSLYDACDYLMWQLMBRTBSVOGBCGGSQRS",362,],["KIKAIIARDDBHDVNNZAJDNYAYGGDBIAIBKHBEPNHJQKJVABNIJRZKOBLEEWMIAIXKROIMHOEJSAXAZFGNRJWUOAPHNNXLMLFMKSERSNVHIUEGIMUAZVLWOVAJTUHPJAOUPLUIVHJOBGAEKCORAQSIMBDBGNRKATHTKPJTWMIMFHRGTIUUOHISWCIORWBKQCTBUQXZNNXTMMCAOYIOTXGHNYJWRWAWBSECLTJVWDIVWVLDJDLCECDECVAGMPRXNRMCTVAABCFNQNGFESYUEHRXNUYYUMDYKSRJZWSKQSSRPBHIYDNFONBHSXKNOEBXFLEDNVMQBMPKDVMPIODIDPYNYMKXKYBAAIUOCCQJPXTHEHSZNUPADRMPUECKTBRWQAGSHQYXSPFXNPXIFCKYVHKUNHSLVVNIZKWALJFRKSCTRNNTYLNLWZIOOPDWEEUVIHLNILRXWBQCLKMZDKNOATGFPORFKNLYKTFILNOAHHYJALEFTOKTURBSTNQXMVJBRKOXSANIWLUDPTKWYGJFKPXXCPXMGBNUBSNMMSWZJXGSUDFEHFMHLIFZQSVIKCXGIAHCWXSQJGMRCEJMPOLNOVOJATEPGDBLINBLMEFHRFNZJUAUOOEVYIMFUUSHCQMLOIJBWEHOHLXSCUAINNLSHYDNIHFFHIICAUHSFAVVPRUOHRFXSADPHIIHZQPVVEUCNQJFOWOOUVSUZGLYKREGOCDXNCGXYGVHRZRFYZOUSHLSBQLTQGUEOKPLOBEYMNCBAWYAFIBNEIQVGPQHHBJBDQIFWPCVPXPRGHDVZXGKWNKQUSRTVFBABROCRPAOESXLPPBJWUCEAAKGUBILSUCUXBKUIHOQCUVRTZGZUOVWFXYTDUXPNJZGRQLPMQBORVHDW",983,],["UHYQQFAQYYALDCJWWMQKOUUOLBMNECBEAEHJKDNFMHUAWBCLDEGVEIKEBECJEGLDUBQBOCIJGSDLKOKRLMHXYVSXCYXIMAQJVNMCZDLUDJSXSMTLQPCFNOQISIAJURAUFCKYAZUVEAPOIWZCPMEDEORIURRYOAAWKGCBLKZMAPLGSDTIBWMVADTWJEEBNHDWEMOSNXMODPEEIAURSJPRNZVGWUKXGOUQGROPOXEQPGTSLFVEBTRNFSMNUZXMSAHPOCYCNMTVTFJMLAWMSXLADWSCCEEOWTOGWQDOBTLKCQOMYIASEYSOKVAUEJJLXVBVMEENBHZCDKDLDXWQRICOSLZOXEKAQTNMGEQDFUFNWGJLEXBYOWYWUOIECRCNZRZOECEYKLOFCCCLJQURBUSSSCMNWCXCCAYGAIKJDDNFQKIWTIXQPCNSMYBJNDTQYSMNDSVFOSONLMEZPIJNLRZLGKZEBOMSZIRJBHNSVHYKAAOOMFPCEMFXVYFULRLRWPPNLGSVVFWNRUGULEIIGSYGDOYMYUFUHRJRLSTPSEKCKNLLNGYJLGERVXLWCCAYNHTOXGSNPJAUNTIUTACDCAHRKJPWKCYQHGSQAVXHVBGMVIOKPABYLNWBVWQHIJC",904,],["HHHYSHUAFYLDSQFVCTMHQTBXHQFVLJCCJHUBSHWVTCHWXDYAOAOSSEYADBHJUMDJDQWNFWMPSGDPQARROFHSFJMSNJWAKRNBJAOXHNTHXOYZZIPATKHIKYNVFWQGNOOLJTLUNLHQOXMNPJIHPDGEPCIIHWDOLWNJLNWZAHWNQEGHXHAHPMSDKULBYGVTZKHPJHURLKZOPSIYLDVQBXUGAMLQYHDIXFWYBVODDAQEGRUOHLPHJXUBEGJIWOHEYKOWGGOIGADAHAKLSDETFSUSHVAEFOMMNGYCUEDXRLBESIJKWMFUDHFKHVSSLHFBWYNLQUCWPNWRSAXYEJJNWVHLGNNNURYHOEAKGTHZYWCNVVOSHDJILWJFIDHFMKSDXFOXIYJZOZZKLXOTJLFJUJIDWQQECGTMSSFDAYKBHBZICXCCMBHYSGYJRRYBYFCKHHNPZLHCRGKPYRCJOONSPDWLRSZVBNCDYGQHUXHFRQJLKJILRRODIJVYTAIVNGQOIJPWOKKWBWDEAGVTVIFDNGAZXOWYTPHUOBJAWQPSWYRGXMFKRSZXMTBORXNMPKXFEXCAITTJNAPVKTXPBDTGASKOCYKOIROGUIONHUVPCFNWIKOZQTYNFCBYNIPAIEPFEKKQNUQFLYSDLMAGHBCIXCUHIEHNXHOXADJXCWFSSNHGTYVLCMIKVGACRLUEXZUQOUWYZAYZBDMQFJSWDWYYNLZMFQTYPQRRBSIRZEWHPOHQGDAYKOUARSUYZYGLUPVZXUZNOBFVSPOMRJTGVHYIEWIRXOEWCJIYIMDYGEMARDRSYXMSMMUGBTMTCPHDPTXXEJKSJ",113,],["EFWDRKRSZFLKWETWKWVNNONYGXXWOSILRJ",96,],["TGCGHAJNLCNIIEQUJBBIAAVWIBDDYJZGELOQYQTGKSBBOPGKJJCDOGFUARXDSDRQLZMWBDVTOOJZXFHXEJASIYPAWNXAXUMKWQRRRFKMRAFAGFMDHTFHTGZISVNKMDGRADZHSVERETNZXPAZKIVSPHGVNFFITVLGZDKMYWYSGAGKBQVMLFMKNIALMRLFVURNIOVFKTKGOZZUZSIKIFSYABLBOCUATOOJTIQMWRJVQNUKCHVPDZRWQIOKQMUAQWDDPMFFBBLZQJYYQCUEFEZWECGKPWCNHUGOSEBIMOFZSNBVQVFWZNJNDUCOVQAIYHFOCXTZKTVGHAVDMIYRPKWO",164,],["GGTFGTZGZAVZQXCHFCVHRSZXWEBZHWXBRZXXDPPACFGZIFCETSVXZEZOZZNHGAGETZMYSDDYWPXQRWITNMCRCICNPPTHPPKLPJLLLXEPOLDUPEJVKPAZMNPXQVIPVPVNZYKZVLPBVAVLEKVELTNNZWDRLGMCHUVURLSHKFSZABEKTLIVPPTBLBPTCOGSLXHRTSQJVNIVENAMGXPLPKXNQLBQZTPIPNOQFTGGUNHMJFWGVIJTZHECVLIIHRVODWQPZXRVALSMTZJJRJDWLCMLNTRJHLTIICGNAYESLKABSTEINSYFXZQYVLFRHVSDNDJKMGNZZGVHDRTGMYLZCGFLRXRMKTIZODOWMQAGKXYKRJIBRCPBNGCXLHPGSVFITQIZYGGRCIBTHGOCBPZYAXMCGCRABUDQBFJLVUYBGFFPTTSREGBCNCGPLZMMEPCVEWIGGLWPGOHJOOHFGTZHXJPYBQMGLDHPTSLQSELXWFQYOIFNEQPEIKQGXRPKUQIMRCGVJBGQBTJSLXJWYMLISAATNBJKLSRVZNBZKNRFWVFUECLSSVQJCPFTKXHKPVPHDFMFZPPGTNXLCXJPIGEPPUHHHIWNEVAQDOJTCHDOFBBMRKQODJKVFVZVGGSJFHMEADMKRKCVUXLGUBGDIPAYTWGCYTOFRSJXYPAQRLOIMGSNTBELCZVZNPOHNDPPZLPPCKNTCMZWZIABQCNXONCWJYIYVEMPJVBQHDRUDZUIKJUKMUVXRCZSCUODQZORVJGZMDPTXITADALCSTAGUVUKOPEQZEDVYUABOMWTNQISYUZRSVHCFBUQLVBMIIRNFJEEBZPXFADNQHYHNHEPWKYAXICYESJSLKGATULSNOCXUQHEOJYALNUZJOZWAYBXAJKFPGCVHEIRYNUUMEZNDMNXSYEBCJCTAGGJCSYPTYQIIIDNXJKXVGSTHKYHDVZPVFSKLZDTOQBAVMUTVVOIFUECGMSTBGLPHWYPDGESFISQE",730,],["FOTQPJZIOQPFOZQNEWPMTRHXIVKIKJQMKISDRIFOKDIFIJOHPWOWXWSZNXSBIKQGIUNATKNIVIDBVQRZVLPWLHKAEGWUAROSZDEUMMVYEXRCDNGHBPJLKDIUIPAYAJQPDETEVCURMINWJCKUNYNKIUXHIEXBCTABQUIPBYOVNTNUOYBVADYBXQJVVBDPNZMRURUKZTRMDDRUOHBWRAGYYEZYRMRRVNJGLPQIXESQOXQEGMPWTMFQDJEJIPMOGUDHLNROJFPSEOTQSGYHIXKQSZYVVWCNOSPLMAETQILBUOJWGXTHGNQTMIEZJBQPSWUHITQPKUGXTHPHLSYNABKEXDFBTRRNYVBZEBLZKIMVZXCPQOSZRBDEBHQBVKRHOBJMBEAGRFLPOCRLAROIIZOBRNZGVNHHKRBBIMGBWERLMCPGYYMPJBENPIIRXLVEVQCRYMIWPDQHNSEEYKOPKOSITVXLAAMPPIUZSMBLQLBMVQOYZIWYNDKFHSAVBSQGWNONMMBMGHGJROTZBFERIVLUMISLIDYPHELMNFDAYIINQHQNHQLZFZRVVZWKARJJQZYKVVBBZHVYPMPXYAIQOCJZRKYBCGYONRFIIJGTJERXGDSUCWOOITZGQSSGIZRIMXAPGOWBYRQMULNCXNRGNYTLLLBSFCMWHGHAVTLUXFASJVIVQDRAUISIQAIUMTKNCPXFLISRXONSZLMVRTNRQILYKLBEDXXZYKNYPHUBKTCBWKRFLRORXREEETRVPPZFPXEIGDVCTAVDWLLHGXRLEDJILPBRGVFCFZRANNYJXQADLRBKXAYQHSTHMSNDDUQULNLVJMASFDNMORZBVEDMVFVQBBVLPWDJSZBCPCCLLPTKHGTLKPZCBKKCGXXLQJIMMQZKARBJRRKEHMGAEGCZ",378,],["QDRKMUIQTNPUUAXCJADTKBBKPZYDKLRQAMYYMVMJMXXPRDIJAAHZWULEKGOBOBQDKXYDXUUJCHEBWQFKFITBECBIDQQABHLUIXIGQHUPTWCKTKAJNZREBWIYOZDTOPJYEFEMTIIEDACRIOQNEHIMOKWEKDUOEWNWRQUJFRGBNINHYIOTQFTCIOJMULPEUQPOCQEKLIHXFWJJFCTUWNWLXQQNLEFUXCCGOBBXZJYIFPKDVAMTVDQPWMRUMSCCMOMWHTTTJXXJEPFEFILQFMXARLECDAZPOMPDOOHIQFLVYHJWYOGRSLWEWPOYUIPLXFQEXESRJAROLWMUUKORLAISXQWSXIVPYBLSRLFOKNSEKEFOWQRUMORUPHWMTNTOMBHTLNNUIUOTUSEVQBWANQMPQOWTDJPSWZKCXQKRLPLPRBYAZWVXNPVZNQTMASVUDCBBHIHCOCZLUKNFMLNWNRKSWDJXMOKIBXAWEMEVSPJLKSDRHICWOELRZHNFRRGDUCCTMRLQZBIPRSAKXJPLPIQNCJBVOMZSHADNIDMYRRQUOOBIUMQYBPCARSMOIUDPVBGBQGOHISVBAYLHNCXLJGVRXYVUBBRAFHZZVTFAZELFOWYZAVCLRCJXVGWQDUGUOIBUSTHSILGFUMVOOJKAIIXTVBACGEWZXREHVQMJHNXNDIPKIRGRFLLJPBMEXLBJEUUGCPJOXDRENARTELHUPQSXQSMNOOFGDLNNJJEFBSJEPNJWIYMLZKNTNVWIGZVTKZNDFGYLDTETMKEOXLTVDOUVLSUPXSOXBLWENMDNDF",419,],["DCCQFQGVRLRUWWQLVZPVIFALWMOVEHIBAYCBITOLPDUBUCLISWIUTOZKOGEYKHABOZVORTVVBHZNGVODLELVQSEDVEURLQOLGHIFLLHYHFSREJVABBFDPEEKUJJFIMUZNKOMKRARRDAUTQHWXHNEPDHMZODDWXHTJSMYHOAZHWSHOEOOHCIQCMJKG",82,],["TRKWTXZRHOEJSJIWHABFXLSHHPZVZJWCAQDPBGOWXUMPXTWHVPOWOLSPWSJTWKGVHGOAXTXTCCQHJMWPJBSLDABUJMGXIPVAFAFUXYXRQAQWYHDKTXYIQBLMKJITPPUEAALCIHGIOMAWQMDEUGUTVUZMHBEVBMMCKTYBFLDKJILVQLWRTIGXTMGQZXDPDEMXMDCHYCIUPMASBRIFMLTEMCZOTNMYZHWVTIIYQQPHLJJTGJLDXMMYHOQLIKPJQJXOKGZBWLKBYFDRPJLIFACAIFBEWZMVVMWMOFGXLHXQZMYDNKRLTECMFQTNWQLQYVXNPQHMFNGYLSUOJFXZHOEFNEMBDNJNJPXADVPEOJQVRWFSEKFVMZKXNPEAQQHVAVHFDCTFZWBZTQTPMQOHXIPTHPDOGPDCBSPATZNXWTUQMGCHFIYCYYYKLJKINHBHCTWZCCGJFVHTUOUWKLTVVBHITYSXBVGTEPACIOGAEHVFTFNRLDINHSWGJRZDKVOUPGAUAMSAKLIQAGGSHOXMENGXRTXYVREPAAIFCPBXXYBJVEPMKDUOSJLVZLUIDOJLNVAXDWFRELKNRDXMUFWBJPJHKRHCMYSKVIIDFUAOITPYVQMOHYXQYVHRCCBOIFVVGNEWYWZVUGQGBICRUMIEJWFLPQKNPWUXDXKDUETSPEKQIEZYRJDWDQMKWVBLGMPKPLGJXTVLRZQIUCETTXFMCJGKYCTPCILRWUSRVYICGXBZDKBRPTFJNYSRELFTRNBFNGNPDFUQZIITQGDLQBPWYTNTHQVFLWXCBFZTSCXFPQINYPELJEWKELGXBQDKEYLBVQRFKBPPVSDAPEWODLZUVTPNNHORLHGRHLLMCOJILKGEOOQBRGPHETOIEUVZCABNNAPELURMOLAMYARZCRJVGPCACVCYYJCBUHKHBVGKJJGOHXSSBPZVFMUOFWJQXCTXRFXFEKHPRTDVKQEHVKBJGF",574,],["FTYRAVKFKANKKDNKVJEMHFAVFZHECCHKJHPSQWCINJVLWQKVYEJOTTTJAYKZZRGWXBVJUMDTJKVEHZFNIKANCHMFLZGPQLSSHJINZYOJNTXEGGKUKFZZKUUHHOZMLEKDEZHYHVEIMFHCDTANVCKZHTRIONNCGKTOFFRKTCLBKEIHQENEYOVIALHHOKIDFRXADWQAPIXKDGZECGZZSIPMGIGNJDPKXWUBKONPHJPIYCXFJQNFNLYHLJJHWGUDRYXNUDOPENOAFFEDIYRCHCYEPMHAUUAYAVKPRQQOOGAJAPGIJJPLFSTAOHZKKHAGJKZLAESKQNWSXQBKTURFVSQYFFCGWQEAUEYAVBAYKVNDOCVXQQJYONSFFZYLIINGSJCGXLGJUKRSGUWWZQYHLQEEGGOZRGLMBWGEIQEEHWVJUTNRWQDKCQPBJRPBLKNIREARRXHWRUQYEGBJUWCJEGTHETAWPBPCANMJPISNBKIUSARKMGKRJFCIXZDEZDWVRUKYTYVEUDWQQBBNHDWCFYQPSNINZLMZIKPEKDOQRRGHKSQLNHHRECKGTACZONBOWLHCJRZISWGSSZHMNSIYERWUUXMJCLEHDFTHXJNBQQOLUDRXIRDQXOOVUXPCNRHKIEQVPXSQGVGZAOCWXZLJNDZGFUBSASFQRNTZZLUYIHHEOHTJGMDOTRHEHDHXWZNCKXRISAJHUPAAGWDRSFPZURHAYGFMNZHZXZCFHPXKXBH",0,],["NSTLOYIQQQOQBAQWRHRQMAQZRTBTJTKAYCIALHJDAITLGQISQYBIPGTKNCHLBHGQBQLQOXYONWLLKYISMPRQUJFKFYIZIOMOQARISAMIKDQLKHNXUOOGNHPCFCNZQULIBDKXIPFPSERWERXVDTVSLRBTZHHSQXISYTRUQCOLWFRGJNODOLAWZFWZZQEXDUVAJEMVLRCAOCVHFFQJJRLBYYIQBLCBKRQKIJRKQTQNWRMYYBKZFUBLNYGQBNATDAKTXOOTYLLDXBXNPQRXJDENYSQTKKZPVOCNSWYMEEYZEODZYFDVMACEJVLNTJOHOWYFWNGMGEQDCYZZCRANOBSALHGYZZGJFLPIEQHVZQXPAIOQVLKGLHTQZGPSREEQFNHPJTBRQLAIQGFTBJJAYKASNBRXPMHNL",727,],["TTTPNNNNDSGMCNYBRMWFJJGIIGPDXJXMQFJCSAFPOIGNSXVYFWWJEPHXZCFYEEHXWLLXQJDXEEIOPYDUITJNBEEXYHBJVXTUGUTLMWAALGZFSLBWBYLPEPUQLGMDLFETLPTNEKAXADXDFREDESMRFJHFEYZISHSMOJSKLTTFGBTZSPTYHBLXMUCFMXFAMNSLOAPMRSUUXICIORFQEYRHVLLNIYMGIBLZOMQLXMXUTDOTNOUMQSTATLMXFEACELPJZALWQFNIMCSEMYXVNKMBZQCQEYUPYCGRUATJCNGTWSMVFTRFMKQYZRJIMIXEEQMUQIAMSPDMBDWFITGNBLRBXQXUHYINXJTYKRMFJROQGHOMZBLNWTPLRWNXWLJJRXRBMFHUHJOHCCLPDNJRTHDBPXSAZLPMIGOFNDGUBKLRYFQNZDDTNEQAYFXKLVIYCTZOFFQXATMECGKWBBNICPSXJFREZEEXNRZSONVXVSQMLJQGEZWOQHCVBYHGXVZWMGBDFFVDDCDWXPAZLZTUVIOUMHEGRDZVQGNGGGJYJOOESKUBAEDDWYLAJTZEUFPNHYIFCBBVJEKPHIHOOUFHQWTUYVDHX",336,],["IIOIXIVOICCALESGZOSNVDZNGBESBPPCMVPOIVBGCNOOEMOYZFPMCCAMOXGZEOCEGTTHBPEEKJUUKWQOMFLFVJLXJUYMKSTSPPFYDANUTPMJXARIYTVXFSKWOYMSNMVGNMRWTEXNVMVKRJPRCBZYSAGCOJWWSPYXKBRFGQVHYSAAVIGHAOORAFASJAMZDKCBGOHKIFKEPAISOVIKMNVVORMIRPRCSOZOBYQIJJWPAXLCPNGYSQZUPZJKBUCEKSEWUPLOSSGAVAYETYMAZSUWSOPJGZGASZIYXLTLPUYBNFYXTHNREQKOBJWSJSUOOCILELLXAFSKFIJSYODSMUKXFAMFZDVTZHZHFJEVUPOJZVFDETRWYLQQTQMGZDPSFQLENICGDLLQOAIDHRQBUJJKMXNNACGMGNCFHAGBFVXZYWOSVPETXNFSKUHVSHTVOJXUSFYXPLPTLWQVZMSVAVSGRLCPSZIRIYWWFFUHJOYSIOMHRKZJJIISFGAMWNMMPGUSGOFBIXMYXCHOPCPDTEISHPGZNGEIWHQIPTHCXOHHQCHWIBOCOEXDXJGPINVLPWFRNXBDZXRCFSAL",646,],["UUAAAAMFKAHBZFZHMVUAGYOBUUKNPLFZCLSONKNKLOACUCAGDAKFNNUMLUWOBGUUKMNMIOALYROICMMQLFRMQICNUIABALIRUKWVWNAFWMVJUXENKOSNSXKAQXBFFNSAJKUORLDZDMNEHQXAMBIIIXNNCUGSSKVAOWYKWYNIJBMLLQDWKANRPIHDYGOMUORYDKNYZHRBNOOGEQXKHPWQFKQSBETEGGWNNUFQMCUHAZNLXTIMFGLXISGVINMRUQJOBFPJXNXPBYRCNECSNAAUFYYYNWRUUUOSUMYLTNFDBYIWZHKEBVNAQGZLXSEPGHGGNUKYTKLLUOKFVONUEZSIQYTJGQLLZITTAIIUFZGAVLAHHCCVQXQMYLBTELVXFRZJXAJWVPQBKSUTOIJCNTZKSVVKIHOQNKSRANNZITJXOPUKNLQKKETWQPBDOIVBTOTLKBBQMFBILOISEXAYOERMCLEMIOFNZTOYWSIDAFFTBOEKOGSSVCEMQMYBTCODFFETAKACDAZQJIFMOHDBNUVXYKULNWZGDNLWRTOFFTLTUWMJFCZGYUWFGSQWYMNHBUBMTRYHEGZKKUHOCDFDWGCAGOHIOIYMOGZOBIPLQPMZGQCWIZWSFKWAKCERVBGDZYZKZWWKAJMVELHIHADIFDRQCYDIPZNAPHMKZGHPCSWAVBCZOLQVYHMJGBDPEUWQKYRXEAPYXVJXXPGSCKALFEFOSGADEWPTKOYVPSHWIFGYJKUCZQZQLUYPVVRGQJJVQKXWZOWGGEETTKOBBMTMAQQAOTQQVJODXFNCOZSIBKULBEQCSQGCTBOBHPSSTNYFFPHGIFGNQVHKVQKWITGIUGRKGMLHXFGEYBFBPORQXZAVCFMDTOPQOFQUHVESIBSRFJMGLQPAOJLZVLEFFGQYVWIGCAIXLATHGHJCLYBXPUJRUOMMPEOMDKUQXRSQYVIMSOQOBFRVIYCAQQNOCJFIVEUTYDIAGE",105,],["PSPSZPBQVBDWZOYPKOEPSQBUTKOBUNPSHWHVMPJJACJIPKZXXJIRZSIQUWFEIJPTAHCSSHZQGVTINZELBVFKTETZQVRTNATBSXVLEDJHEVKFHWXEQLHCCNQQVWEAOZPHHNQSUATVZNKVIEPCHFVWAKBRPPXANRZVNBEYTAMPGEUAYHEPKYJBAJJSOKXYIOYWRZSAWXAZYZSWSONQEYGVSSHJHBUPMUUZNKVABWSZKBKEXHVXFYCKCRJYTPHVCAXVHAMHTLFEZYFLNNOTSVJLMBFHCLGVYVNJHEKKPPNMSBCSSDWHNDPZKBXXVWVUZTXIUBPSILJBFLWEUEHVJHHWCJXDREAETRTZTXADQTZCHBNTYOIFEQNHOXUDXTPWNZBQPUTXSKWBPMIHNEBNEENDHCUCXKUOBLVIOMLHSPPGAJZRNAODPBFVVWIJWQXVVMCYBLSCPUJBHMAKJSVVBHBQMSTZNSWOYXXXTSFMJXEQHLINQXVBZORGYJMAJDJCQOLWMSPT",369,],["JHJVOXBUPPXXINWMJWGAHBKWMPJHKHJCNNMANHNPVGYOBMSKQTRFWJURBWJDUMVYHBNUKHVWDHGKVENGGCBONKDGXZPUKWCQOTZFZWXKVGMUEVTPNGXCZQMVLIIAYIRNRFNIZPBBTXVGQLUSRMKJJXPVTMJWBXXTFNBDPNMESAIFTHMXEJYONWNSXLDMZPNMNXKTEFJSNKXSAYCNIJTBISNHDBIVFPZQKJQMILNPZQFSQIQAOXKQMFEEUOBODQPAWPXKFCFCQMIZQVCXAMHSKWLQKIOQTKVJLLCDRHWDKENMCJHXJURWZESCVPUXWZACFZRZUUNFJKXTVOFSTKZWBQPGDZCVXNPZNBVWILYJOROTDGSMZATNDNJVFGXUHRWUZBWDTUNKJJKVOLKQBBLOHLIBVSCAVFJHLDFXBBPRXTEXIUCESHCPDYTPBKGUZLYWEBOPTRPCKWHGLISHYBBQLBLHVXLHCUCMOARTRLFIFCSPACJJWOQHHALABGTSSOFEFLDARBNYCHJLQVBXCWCNRBBHEQFXODGLUUSXTEQYOXGHVIPUSBRMEQAYEJZBXNHTJNHNUCNBPXLZMJP",5,],["KCEJXJYUPDUPNIEJEVRNEMXXLPKNKKWVVIODUXNDMWHLEBOUWFGHELQOQBQJBXFHDCNKDYDKRRHLVLEDOERIZUMHVORPCNHXKJRFQJNZPDVJPEATQMCGWUPQMEWYXZTBJNVPDOBTCMWCDMHJDLJHZBUJXUKBCXDWVWUNVKWDQJALCBKUQATXFJAWDAXDZPVNMKBJEYQUJVDMFOFDPHKZJMBDJDDBNMHLNVBWQJBIKVKOCQCIOUFETJQSXBXJCYWDBNAESKJBKHJCRBTFVEDBETQNAZOTLILKKMEIOKNLNAUEVESTJUZPEQGBRJKHKOGZONAKBFQTITZKBDYJOASQZCJQEDPBIHEXTBHKLWNGXPVPXXHHRXMAUFMNXEPPNIHKWNP",421,],["MFIRIRWHMHYKIOTCHDJNHWTQCFQHGMIJRGMPINGFYDIXFDRZODJKTDGQLEEFMCOEFHEIXGYKSQMCLVMSLKWTXCAXAQRCHZZQZDJHCKPJVHIKHHFKNDNMZHVXEQPINCCJRWLBFMNNNPYCVYSQGFVZUIORSAWAQXHULKZNGOHGKNDIWRMUCHPJAWAQFPVMBAOUWVECSSOIDEECTQANXUWUVRZYHRBIPAHGMSSSCHZBQHGAUNDVAFSKBDYKFSKZHIDEHLWGZIJCQQGTFIIKITLPIISRWMVLTBSVTSJGAVCRPZQFDYMGHKPJAONQHDMNMQBLZTJNYIQFIBRVWSOWCHIZDTUMRKAVTMRNEMUXJDZFQLRPKCDAEKBQPRMZSZINQVDEIIDYKOAPOCOMCQETKEEPJPMORJQSEAXQCVXSZAUCGHCGSNARXFJAAPHCDASPTCZXCHWVRLESGFILDNWTOXYTHPVOXOAGNZRMWOTDTYYTAAEDVTQVANMLKMWUPGIMPUMYPEZQJASBOFVUIPINBJRNWZBGGBSWXWYIOZSZVULFZWXJVKDFHXLRQXVTKVGGNHMLRLITRDDKMKXHWVTUHVCCPTPXWXCKJNCAQVIZKHAYDKUKZNFWIDBEYVPBAYXOELAGRWVRSWDXPEDDCFAVFGHWETTIXDVUZASWRSIILJSPDNIHDQZJKHHQLBRPCSDIYOCZQKZRXVWRFENDJQPSFJMOKXYFQHGWEGJEFYPXSQBGZCQACPWOGBZRVNKKUYTQSXXBOYDHXIEXSYFGCQYEDAOVJAVLDOBCCXKDNMQRICGYJSLFUMEOQSBJKSUK",659,],["YUXUIYCNTMXBRYCYIBIGFICCYGFZJETNPUTUZXEIGICMJQKKAEMZTBLIWONERGOUIGHBIZJSMDOFHGDHBADWGBIKMAGPZGZMKQYBCLLPKPULYRNWYCRKSJIKJCVCPPCFPDURPJHFWMNOQMPANPPEGOFOBCUSEOJZTTCHYKYNTQDEICZSGZUNQHGANPZVITMGDTILOMHUTNCPRFJDVBSAXOYEROBEHSXXFGMSGRIPEDADEZDJLOTHEGFELNYOHHCEVPFBVZINUSHWXMMCRMKVZPQIDFWPERUXGDTZTYMDYPWHUYEBOXZURNJWMVHUBQWLCDIOHCHGZQOKWQVKVPYFXRXMNXQBIZHETRDRAFVRCFGSQAMPPEKOVISAYJXEEPZDYGBVQKUSSBSWPKJUYMRLRLEYWTPAFNW",563,],["JEFWMFFDMUWFGMMUPALUGBRUCJDNZYNIIYUSAINTPQQPQFNQXBFPMMYMXBPBJAMVFXOLFPFTAEIYQKTDRIWRVDHZXVFTLAOVKBIZTCPKIZARMBVHJRHMXWVSLGPGPNXMCAOISVOOAZUKBAQJUIIEFUAEXVEWVITWQRKIEMXPHSSKLVPVKSJCIXVYGAWGIUMHKFKMAEWWJBUADIWZQGIZANIVZOABBPCRWLUVOSMJRHCAUKAFGZLBMYABTTYJAVETLHBXHCBTFPZLGGAZNMTXKAMMMXABNOPIOZKYNJJFMIKWFIKNXUNTPHNVNOBLLTSENDFMFBKSSJOWKMPSBLKGTZSNUVFLNVIHVHHSMQPFSDQEKGXUDNXUSATBOLUBHRFHLNUWKFNRFVDRAXNGLIXGHNNUAFVPWJQESPQXDERHZVKHNRZBDYYWQQZKSSRZNATCNKRDEFREYDBUBEDVGWFEXUOKEDNDMKGIYMPBLNRYVLELYHELNGUWIMHCTSBTAXUBADBOSYHPPIJOBASLKFDGURHFDCYKVLFQRVQAPPHODHSYQXRCYBZWWBIKTZBGXECDGCQWTACNCXSIONWNJQYNTDVZNWVUNUEXEKYILIDMBXFXXJZMNPLWXMQEHLPEHXFBRTTAFBLNVGVYNFLXETIFBCCFSFBIOCIHPACJJAUTSWPXCUGWCYUTFBEVRXQCOOWRPXWMBPTGMAFEFWWHZJNKUKRDUUBRWIXMZWTKOFOBLXZBQQPNTWUHLFHSCODUCNOCWJEVBEAPFUGUJGGVDSVSIRMNCTHPXBRHDRNEOLSFBDLFIAFF",276,],["LNWSRJMLLJRWIQGRGRRSMRVERHEFHBXLULUSIHGHRMMNJVUWHWJOASJARRJUDSZRVIVJQPAGGXYGLDWZHPAQGIESSFSPDMCEYGZHACRFXRJXPGFGLQDNBXFHJWKBPBHSGLMRZMEQSJVINKPIJBDXUMAVOGQNYAXRMPLMNFIIVUEJUTXJKZPIIEQUKJUHDXQQVGHMARKZONJMRHTHTNGLSSBNGCTRXISHTNYQDFOSCROVVKAVZBGHDVXYIXCRPDXXATOVMDZXFXFSDTUVOIEBUXFATPURNATUEGQVRVKJFDIUIKUCTAPNMWARWLVFNNUIVDJONTAHTMOBSIBZISXYZYZZLGJPFCZMFZHZYNVIZMDHJNWIYFXBIPSLPTJBUWEHTFHGKUGYAMWPKEXEPJSCIYOIKINOGCVNOHBGKWBCNJDBXPJXAQSOPDPBLZJYYMXYDOKANZXBKEUWYFMYAREWGARKMJZQBAIEEWPBEILTGLAEQUZHBBQTWJMBXAYDGHADCSHKNRWTDEJSRPNEXYQEXBEMLJBWPSJSNXSGKBXWQUCDDCSVPXLXNAQDQOAJXRNNYRUMCYCNYCJCIVOXEIEIYCJXFPWWYENNTRPMYISFNBVGXGMIZEZWFJYYXIPXBVBFAXYUHCQRCQRBSRDPDEABRESOKLNZPHZWQCWVPAFLNZBFTCWBVQMETIRBJNRKEQLXJGVPYSCBXZEQCXXQXFWTSNDGUISOQMAACTKNQNFTAYBSZAGORKMJHFNPAFGFRECHHRWBBVHLJMONZGCHOLIKFIJWLYAFLHXROYRCCQVVHZEYARPEDAMSPMXMSRUHSXSVGYHKBNDLQQQPZZXHELCJETXPWIEKNCPWUPAIKBBSTDPKNUPLSHWEFDZRAJGBXBWCLILYFEXPRANUWJKRKSVUZHHTRNILEQAIDRLSLMJEMSLJSIGHGZFFKJAWLPQBVQSTVGLVIZBVNQJCIRVWERWZVNSMWPXJAKVRPWV",891,],["LPFGVBPWLBTZNXZRZJFWKYEVQTHAIORUKVPQNMPHBKNNNEXRISIMVEQQNJOAEBGWXHHIDJJQMFIQTIANGQTXSLMHMNWAVFMQBZWRDBEJXXYMXPDXXGSTHSNYPYWJTTWCRJMVXUOUIOXTIEABZBEHKABOZTTESAHJMPOCMENQGJZXYWFOVNKPXJCHZXXCTTCLQBLDVENQCINNDDGVGEEORTARISVSGAFIXUITAEIYFBUOJZDQAGFXBNHDDCPOSIWXKYDGIXPITHRRZLRYPNVCYLLYDPGUBEUMYFESGJXHDKHLGYSGATGLPWYBMBRXVUSISFHQIEGRAZIVXXGBASCQMGWVIFJBAOJSSJFRXLQKGNQGOSDDPJIGUNZAODFJQOMRRFCTKOKFZYJYJWETGYJJFFQLXSTLUEXKDIBHDTZKXSMHHZCERQKUKHZWPVXJWLQSNXRPXOETAGJVRZRCFSVORJILKYXOYDZJFUWKIFTBIRWPGIAIIPRCCEXBCNIJQWETPZKQGTYOWVVNCKZISGEHDJEZWOSZJNJZXBBLLYBHXOCYNOUCHZLGBJUJBYQNRFRMKBKAXTXGMKHMUWIKVEQOIWFCTDYLKMYWPIQUYAMNDBFSKMQFQMCVFSSTWTSLLUXSZTHCCXZRPBPPQUIIBHILDGNCXQNZVDZATVZNLHGOYSBJNVZLTWPASKSTZPXLLSYZRZSOMP",820,],["PAWKZMLCTRBWMXWFKJFAPZUMKQZMEOGXCPVKWKBQMXOSAVRENBIKSYQQWDFWBQYFSXWPFVEXMXJWWQJUJUXOV",244,],["LLLABLZFMXOUAILBIUBLYHOMGLACCYPKMZQIXHTCDCLUSALHQWSZLRKZEQOQCLHXAZVNVNMKQCOCAOGQYEOXRBXZLQHOCKXHBAIGNBVULYQHWVCVJPTYRWMOZUPZMWYTCXYRWUMPGREVWIYVDWGSVGLCBGGZCPTGZVRUHIMJVMQNQFQCIVIUUOPYRYRTZUYJSGQMAKEDVDFWRTHHISVRRLNALLQWONVRYHHAUXODGTDIDBWNFPZPGODLOTZYUYEIDHSTMHBIFLWDECHXCGHHUZTSHFIFMSZORMQAHSQBNBRBNUYZJUNISCTOKYOSAOPCHLPBVEOCDTDWLHCMBSMRZDKEIBZNZQRTSVAQVAXAQIDSOULOADZYVLIQURNIITXZVDTKMALWEBCBCAKPGASIIUOKQBJGAURGWTURSXNTTNIAKQEZQIASLVWPPIYWWUUOBRSDERBJOKWQWKVRBMBSQHQMROIXUCUYIJARPSXTIBRQQYHVDXAYZMNPXMQEJVDRHSSZTSXWHCXHPSRPOPMVXPWSLJEXOLUMBLPTELMZUSIKMCWYCUJDFUDSGOHCCKIDYODSMISPMCBVXQAQDMCWCZYGMZGOIDZDATFYFUNIGFRLPLOMEKHVYQUJJKGATBHIFQAWZIZVQYQRPBPORWFJBHXXHTXZQUMALKVNAISKNVRXQWYUMUSNVZCSECTXIUAHJIHYLGLUDKMPTEPIUPQYLNFGZSPYOKFZVRVGFKURUHJSVUUFZJPZTPNECEILOKSRBLLPHSNOMZHDQDRTKICOFIYFYFNMJDAOZXAKJZXJWVDIGOQMENSEHRIPIQKTXPDZDCDZXQBJOPTCGZYPNTEYSYHCXPRMQOOHRNGZGHHEVYNMNILBMNPDY",506,],["FSPSLISFZDLXTDWQUDMVUARSIHNBDRTULWUQUYNBGJQRDNVSIZJEVOCAXINGGDYNBBZFDSBZLSTQLZVCDZYYMDTUYQXZQMUVYZOLOTSRHOOAYCAVNOSWQZUCHRTFHSZIWFZABUKZDIIYAJUMULNXOKECKZOSYQGZKZGYUFDDCBUFZSSOSLGRBOYYRDQBZTUVZNLIQICFWOFZEITASPVUSPLJMLYELNSEOJUU",754,],["JJBJRVUNKIJVNVJCRRSSKXQXYIWUSUCXRJQDJABRJXVEQXTLZMFKAMPAJMHDKRYTGMLQDDJRKXQZJAJVCVDXPZXCTWJNRDNYDYLBRXAUJWKYBFNQXGLDTUXKHSEDEMVDJAOSSDQIJJZCYLBJUCXUKMLFBCPAV",959,],["ZUU",708,],["EEEEBHWLSKLXECUXHHGDQYEPQXLKFIFGTRBZFHPHSQGBUGHEGOOASRULRUSNAIEUEKXOETKIYHCPQYTGCJOXTTUBQXSFXOLPFWSCZPOLOBYFCGRIIOMCDLYTSBIJEYBFRDTOLXSWDVOYFSFOLGNHXGUUDBYTPUSNPCHUMNVTONSJGBDWOXABTSLHTMNYFKKHPCLMFSGEKHUDSNWVPRHYANDMTWTBAMSHTTHMPHFNEZLSXEJHWONPNFSWUUHTKGIEMHXAIKFPPVHPHXCNVIRSYUFMFPAMVSQHYJCVXFLUEZNLJNLVBNOEMHKPZGGOTXLJNTSIPFSCAMZRFNFUMEWLXWUWZSTHWBRHKLUBKRMKTSLFMGXPMIBKWLXOWRJCWSWCWBENHXMLGTFHODEIAZJXTKUCUPHTIHQPHTWEARTXUXZOXGCVPTCUZMCVJHGLPCIBATGUISLCAHTJXCDUHHWCFMYYEWAXXYRDKLWXTKLIYCFESUCWLTMLYFIFMYBGKLRKYBKFGGUPWVDHFGMSJIOHLSJDOKACUDBJUHGJQIQPUYFLQKSYPFQGAFBWPIFOYEBVLKZIVNHCQGCGITJWABRNMBGMICDEMMXJNIMPDBQADCQYSICQRNIJBJYATDYXBMWEIQASVMTNNETJJMJYVYCSALTYTFSUMYLKIDNYOOPYYOQORPMBKXTGTIHXTWBSUHFGHZIMZNFGODVQRADJQCBZPWTUB",630,],["RCCOCWRLCMRXVUROIQUIOLUWQUCSWEMZIUJRHRPRCORLPDUCIPHFZUWFOPUGWEWCSUWIYVNQPCDUBXBHNSCXWBNMVUOWXJSUSQUKAKNRSULYNOROSBBSJAEQWEOQZESLLCIURYJOHNAYLYGYFUAGUSUINGRYPZDODXMJSUUXUABNXNFIFCKPICSFPLJPZUTUDTJKPRZUPDJNAELXJIPGFLJJSWRIPAWKIZHCXARRNAFECYDMABTCGLVSNOCYMNIKFZULQWYUHRWOLACRIXBGUWKUYYEFZXDINEVKKJSODVQQMYGYLCKIMEUTGFADQGNXGUXOESHTLKGVVCLQNGKKGHWGZPGOTUNBPKWDYUPAHAGGINSUPQNRGIOIKYIWFQEGEUTIWJQLLWPDCJKKVHGLDJCVQPJPDYZNHVMSAEJTUGPOWHSNNIYMVJENYNBRUIUUXBUYSRLNIEBSWPJYESYHBNZTIHUZCTMSIHHGZSQUPSIRWMDCZUSTNGZOWSKHQOFEJREHRMRJZZRVYMPPSRZSOYJRBDALLIJPWYZLEHXBLUAZEVZLCHJCWIKQWRWBFAYAPOUOOQOUWLELCKCQBJDOUXSYGXKGWALLBHBPLZPKICAEHMYWDWKDHJWAOVPMOHJUISVVOYINJTNEJUU",626,],["FUMJHGJFABDNHPTZYZUEWMWPDYJNIVTLICNHOUPWWBHSQLJTNVBWXFVWBPATXEJRFQJJSXBRLMLLEZIFVLPGEQAODEPTBCPYJHAXUXGVMDQFHRWNYOEHJLJCCLTWRPEDLNDNQGYYQOGJKDNWTXYQQVBJFJBUFBONMBPCQXUJWIMPUWBVCMNAAZPEQRQWTTAOFJBOJLSRSMFDPDALEJQQHCYXFCVWMPOLVNGSRTEMEGZMHEWVFAITEYCFYSYICJPMVBXTCIROIGNRNWYFXQYWQCWNYYQSOMQUEAJW",313,],["RRWPXUSDICPUDEUDJATPJDPDNTBWTIRVRJWUGMDPGLMPDADOUZANUBACCYLHFHMAEEUJDJNJHIRYLNJYLXMLIQPTEJPUOVTZHPRMLDDACLYAIESZPMRCAAFLHIUSPJTCNIIUNYMPUHKVLGKLTYOBMANMEEWAEDVFBDHMSIOZPUUNPFNUBEXUQPKCLKWMCKIZEDHRPWVOTKCORYBKTFIDVECOBFAVAADVEWMDVOXWSMPDARFXRODCKJEEBJUHKHHIKQHTUKUBZOSQYOVTKVDACVOSIUYUATQIZUVQULIOVKPHZBWIVTRKOJTXHXGPDHNXMJDSSNREKTZNXMNPMTSZSLJNXLMKGKTNBBVCQEROVMHDHJTVJOMUEXVUADNUVSPHEAOHZBRRKLSMLYCHEOEQNZYZQWHJOPIHFGWGDFMJFUDDRKUZPZMLEORSDFKJSIPADYWDJPIQOTUPGZTXNBGJFXRFOYYYXSZAXODMIVQRCZWKGILEQIBJRHCRAWKSYZWCXTFPMVYOTSLDQKSPVXXXLUNULMJSHMVRXPTMTCMNXJTSVAPLXOJXBJCTHHMLCQJANVOALDJITJMMANKWJSDWYCMEDKQFIRUWNIBTSCLOVVXNBDYNGKQJUFJJKGBKUBFAHEAMAXQEASBTNIBHATGQIKKYFVVZQAHZUQIFDZZXZYUJVYVPOXHNSGJQGYDDPPIVENKEEDPFENXNUJSXMRAXKKIAFPCTJGFQVCSTVKZMIXWGOHTEWYVAUNTEROMLFDGAGVXSMWCQEKIVWLZXDWIUFRMCURTTXEJTTPTSTGIBGOZVIXWLXBCHCDVVWGHBKNJDSXMHNCXYPVLMIPISRFUJOYHEDVXTCJBQTYVARDSJRSKBFQERSFRZFDBBKOHHFBRSUOCWEDJYGTULMRTMJFLVJYJXDTYXLGVTGNGOWFZJULZETALTEBYPXUWDOFQLKAJZJCEOMNACWXJPFTUORXJUDMIERLKYURYEHCZDDZ",379,],["LROLZPFTFZPLRRYNAFOFJZQZMZBDRRFYHVWBATJARNPGBQJRVRXUYDOIGCZQZZVZNOCZZOAVXONAWSCWNMMMCNGILPDTEGDBMCFFFVERLFWATBEXZCDONXCYFJCVOPHUBYJZSEZQTAVHOKUMHBCXCTWWRBJOFLKNCHGTIZPUJWYOBWHIPUZCVBYZBMAAVJPRUMTFJUEVPPNNQVHIXAMPBJNAWKVCLNTOUTYGIWIARLDWODCKWLGWJXRNKRJDTNRLLIJOEFNVBMNRDHURTSXHHUMFXPPVNTMSOVXXRVFMMBSZHTDMLCYYTCUBNWQWANUTPSJAEQHFMDRZDESBPVACKTVIXGTZQMBSKNOKZWNLUDZDQFXYMDVTDRHNZWIXNDSYYXPCWYUFTUUXLCGIDJTEXQBFLDTUPCWZUKGPQYYVCUQOOTRFMDQFVZNTKJMMAXRFUFVUVGQTGBUNTDXPKZGFIFPBIDCSREYTMPFCCKLEVFFKOWJPFSDSZZEMZXOKBYSPNQMMBBAKJNGKSYKYOZJHLGHBCKFGKWMFZHBNCWYPHDMANODLMXOZTKADSRMPWFIFUPSGEIUHOQDIAADFPJVMSLSRGZWBLDLPKUFERBQMZYCYHTAF",203,],["QPLBFQQWUFMKXVFEULGMLNQOGLSEJUSVNRFIAKSLSWSWTBATIKUYAWXCAKVXLTENUGBTLZBCFRLIUSFJILRWLDKZZDZLLMNXELGAFSXXUSIMXUIHBUACHAUKZTJSJGRDMZCGAWIBCONOXWYKKOZWBULSTHUPQMQCGRXZQHBEGLSUSZWBHGVZVJULWTHEFWNZTWONLTWNONMGBJANIYBLZWWTAGNIFFSPTWCVSABNKCMAFWZFYNQVQWNNBEPDBJCREYNDEWU",13,],["TOADINJZVBYJYOYYPLVXUCRTLYORBVSTILAHMFTJPWIYHIKKDPPXTDAVBBCWKVJTEBGIVANVAXXLUTLTGEAGRWKXNMTGIMZAAWQIHFDPILWPZWRHZUWZCJOYOSQCFSJMXETKDYSKQIPPYJMTWYMCUIBQVPBHCYOWCUDNDSFYZRDPACTZTKWIDFRWLMGUGDKDZSACJHTZCTPJDADAMLGCLLTBUFELSRVYEEMCYXVCSUPEAIPAVGVFTFXTGFTVKNVFTEFZUQHYAWGWKZYQOCBZRFDBXBFVLZRJHOIVFRIBSSKZLKRCHJAJM",57,],["ZGZGPZDFZCBLTZOTMRCPXWLAZTJPPFFUFNZHFEUEXFKICOXUBJXLPTZQSTCQICBCIOFIZEEZSJDSFNKFNGSPBWHVZXPIIEBKZRFSJMWNAXPFTCZVZTOVFM",968,],["DDRP",267,],["YYQKQLSWFWFLYIBSZQGMKUMWXXVLCPULYOUYAUJVSEZNQGEJJSOICPMVNPONOMHVADPIDHLWXWLSXYCDSKUJUHKPILGNMFF",454,],["IIIRLKTZTZGISIATFKKWUUAOXXFYGIYSUYYLXNLGUJKSNDYAUWAXBBROYGOIQKCHIHOWOLCZMTVKXSYOPZFKIDXNBWBDIZNHMBGLSSIZMGHGBDTMEFYKVZTRHFIBPKIRIDUGHZTGYGBIREYFYZTVVBMKECCZXXMVGBTICEDQBGVOEPIYMZYXVPDOZQRBWBIGTPXOVSQZWR",807,],["MAFFR",250,],["NJNXJJOOLUGENHHWJEBLSZIEWFDJAXZDELCNFJRGJHZGHVHNPOGDKKSJERBFNEBEJHLKLBFUVJVPOTTFROMAEZHFQZPKDEYEMHOAUCJTOVNENPHBHMAJPDPFQEDQAORDKUZEXDHYZUZZXKALOQDOZPUOOIKZUNKSZXIDLSTFMOVCAYMVQUSBTBCTIFZNOSMDUQNTVNHPAAMFBTHZDTJRCQELFCMVYESLEWQVAZDEJDVONFMJOZHUZPNWXJIYGTRQRVJTEZNICIIEELMTXQJASTULASOCDBDIAAUSDDEYVCBCQZESKMHQUYEOINDCWOBDNUPCWMTOGDZCGOTNEUPGIKUWSUHVQBLQVICEWNAUPTAEZYIHSLZIOVMGNQPWUNMXDVERCMTIFLFEJLDKLPJATDBGPWMOFLKEALHOKJUSXOEHZHFRWJTBENGVFRIISWDRFHZNSOATEOZNDHPFYQQLXGRZHCIKXZLXDAVRGTCOKMJOWKWGGRPPQGWWEAUHUWLVZCUBORWMHCLYGTMHFATYCDHYRIBNZVINRIMKPZSVJOFDOMOJFFNDPHXMXGPWKICAKWTEDLSDSFDMWLVHWGUIIYFJXPTTVFHIIIXZNBJXMLBPGWPTSEZWHQSSVZIYNDTGSWXTPPGGAFOLNAKHZEJLOCCLTMAQOSRIIIFPFDJFBBVFLJPYGBEACSFCBNVOJEIUMTHHDDXSRSQLMXIVKRKWUTFGNGEPLBXZMEGRTPKSZHHFDNTIPEBYUNNSZSAMGGPGNGXNQADBAMXFSLAAHOXR",407,],["ECELSCSQUWXSVZVPBIBNZCVNPDTFICRWXDXQQGHZPIHEHJCZFVXIFNHXAGHEAYWMIINSRNXVUBOVUAULFQXDNOUUHMOCPTEJJWUSLXFBMPZSWOVDNOTSPZUBNDRSFMLIWQICQQXSSUICDYBFVIWGNXCVNSSTLMXIUGRTHONFYBRYOMWLNSBPAORCECMNRRFMAKCRNVOHYGQYPNEUMSEMYEKUZBDDPIJNIVTRKEFMMOZNMMFGULTEXKWHOMVMVNLHEFNMQPPPTLYOOCPWGXAEMIVDNFVDWHXBPSERFPNYVCGODERSEFRKSUUKJHMDSZNIHKSMUULHKDLUXIANHDSSZZJNKXNDJUCAGGMUUQFFXQHUHOAHHROOZFOUFTOKKNCBRCHLNXBOMBDGITQEPOFVHWZEYQIEKXUGLOIPRXEJLDYKEGADXJRSOJYMWBIETHQDJAOAOACYONWBXHOPJGICCKCTLMUJSGJQRCEUAVBPDFRNSWRKIXAXMCMHAMQCYRPHBKSBQTSRYEBVRSHNCUTQFYWYHYJEIMAENOSRLRDVMSENXMBBLWJXHNTSGFHIMROBKPUKNNEVMMHXBZSITOJUZUJWRQEUYGMQVFIBXNRTWPWVLJJQXCUEITCJSTCSFNHKJWFGFMXFMSGWTZSYLVRVZZWGJZUQGZRJIEDVRCCQLGCDJCWWUBRHXXGRMWLAIQJWCEWSTPYTXKQIDGXFEDMXCYXIZUGYOUHPNKLDGHUZVCDZZQXDCYCNDOTBWBEISSSTQIOJIGEYOINWGOQWTWFHSVFMSOZDSCOHTTERIBJEWLUDDKTQPLOGUFEJCUUFRIGDEISKZVGXNINLLKTICEXMFYRRCTRKXEEHJPWJ",464,],["WWWWWSWCWCAIFZJYTOWWMHPENYFWZCPWBAUDICEHKUZZHBFIWTMYDCNPLYMCWAXWCPOZPCFMWPMZEXKOHPTUCEANPPZDDRGPDGMKKNTRBRAWCZZBDDACUHUAOZVZLWDCZRETFSIEHSKVTTWIGFGASYHABUNNEWQUJLPFTXNFWNHREYADGYQCLGAOWPGJGATLAFGYZNWSMFTPXLRIGPNDYBKPOKEKUMBOTROUOCSZBHHTPGMCVOERRGHORSVCLPHNGXEC",431,],["UUUILOCZFRFKUFLKGYFDKSATAGZUUBUXWAPGBYPDDOWACLKFHDTITBBEADVCZDBFOPSDYRBKIMGDEWSTVOQPMGMGSIDOOBVZYLPJIYFOGJNXCAIFNLSCMUQFAFTSNQSOAFQUGSBGITBBDHQVUTCODJYVBJSBNQGVYXPMMDCUBMDFAFPBBWTMKDCVDIOBISCJJDJUVYGGWYRJLCLBSMQQYCKGPSXKACMEUNSHGDJGELXMSBPJOAUBQEOZEQKCZLSMELJMTOXIYSXBADTEDGQJRHXVFHWKKGWANAOCYWGSBOJOGXXYBVSJPTOEMFWZZQCRTFBKNYTFLJGYEYRDAPSWMQSPIPZJGLRUJKWGREEFOYGBEVJUONDZQBDNXWMMOXKGGPSLOTJCTJPJYZIBOVRWYTVJIYCGDWDDAZBXDWEXCQKMBPMXIHIUOVWQXJDZFRASEAGERIGDUOUSOIXFMBBIBLSZYLKCZVBFCESWLUASAIJTXBYVKMRASXAVGTATBPFOYVAOMTSUVSANDIFJSTGJVBUVIACGSJRHFZMLMPKNFOCZEGCMLWOBUNPWBOWEGCZMQMBRDYQHLDQKZFWBHIDWFBRJPZDJVVCTCQXWAYBYVFQPJTVKSGMMUISUDXEWBNVAGKIQXCG",208,],["BYDYIY",367,],["ZZHHHEHHLVJHOLCHDPGQVBFQCFKUAADNKRJURGDKKNLBVHHZRJVNGVCLFMVXMPVXXXUQPUZTSXOBDDBEXIDMAEVATXGVHGYKFYWHZNNMIHNGCTPVZFAZIJVMROGQPCZKMIISAAYATXDDQKLWVQTVAJCMSVIGVAXWTXUCLWKTNAQBWJZVHBIRWXYTGKNHJPNVSUGEBOPKLAJRYNURYXMKMGFJCOXRYSNRKMOOJKRNQLEZSEVGKTYTACFGKVTVZOFQQXBRFZDNQPFBGWWSRIXZTXHPCWOPHHAYALTNACLJLNSODLRMWTXJSIYXOQNLYLWWQNUOJTSKZSVGSRXTRRHMMZYAMJUVHVPRITBXNNRYADKJRLKWHUAIOUAFVCNHZMXJTAYINYHNFADMLHOYKEYZJHTBEOMRWLTWTOWMISCLPZBMQEJRGSJYZTFSIDPZAYUGCNAJOSJSBXWYSFNLCUCBGHZJODZVVEOLOFYJUPANNKMHLKUKGWLHKYCDRGQKNIFRVXSDNEXQCVUMXYJGVCTHDYRRBVJMNNPBFCEIXNHQJCDMMNFJVTFXOCMWTRFVZWJOUAUNNUTLTOUJZMHQTHTBDOHCEWBETRDBCGZCVKAUAREJRDTMJIZLOOLAWVPPMXHNRDTYZROSNNKTGNDTJAFRGOFAQEIWGRJSZRWQBVMZUYGCWKJHMDGPBGZKOLJLXADCYIZWTHKWCEYLUDRGNQRVUVKSSZMUYZGOVHEFGQHYRCOGIOHQOEYKRDTKYYDQCACLOCVZSAXTLRUNCTOYSJAJDAUAAUHRRQXHJDADWFRXJCOVNACQREVLBMLMZQARWEIRUHVIGSFBARKKMDXKHITRZXUGCZCYEYHXZHDLNWJTCBZBKGRKRRRIKGZRPGIMFUYOORMIAIIKSHNCYQUIKCBBXCVTMKYFZAAITNEEMIYEHPTKIITHGWYTQOMPUJLADZSITZZBKOCOGVYWJU",679,],["KGNERNRYGPRYECVRZGNLJPVEVNJKZJERZVMAODYRWKVRVCLVLMWGMVHUNNVEQEZUBLROCHWZYJRMEFPYRJIWJWYCOZDXVOHGYWMTOTZBZMLNQFAIGSJXIALRLYWIWLXSFAHGZQGWXUIESBYYJNLHTVVJRCEHUIICAVZREHCSWRIOIWRUCPBRJQCLTXQJIFHXYBEHJRCKEWEXJDPWMCVTEJTIZZWBWJOIAHZWMQODNQYQETZWZUKLINMVZHWUZVVDPFSKBHSOTAJPXEUSTJLPNTIGEXGHVWOUHTVZBXSNMZYDTFLZGWCNAZNBKWBQNYMOCZWIBEQGIMAAYXPCTDTNIFZBPYWFOMPKTSMFZTOVSIZXYVDXDEWBHTIVSTFMSTULBILNANFWNVUYNRUZDNOJZNISZHIRBVVAUJLDIOYFLDFBKDYPGQXTNCQPZZMPQIWQMPDZU",69,],["AAQQIVIPGVIINMGDGHHSUHQJSWQBHIHJXUHHAUVEKXVMUOBWDBZFRPACUXZIKLWGTMZHBVBUPWAPAPFGJBAWBTHHOIZRHYNYUFJCPUEQEAUHKQRILILGFFSBMUWCDVLQDZYMDEEQFELERRHBXIHEQGIWVHCIJNOOBERIPPAZEFCJUKXJRZUHMTJQZNYRQTFZJVWRAWOCUCFIEXZOMFRZPQUDMSIHATMNHJBIUPXKUVHFQBFGSHIXHPGTCFMSAMNJGKNGQMUIZDNQZSRDGULPJEKEQTNLJMLZKMJYVFFSCRLTEXAHYCFXDBQUKRTKBPZHQFECROFTDVRQYHNESEWJIBPXBHKMDIUVBLOWGVOGRDEALHSDYJXDLABNDZSXMSOXKPCSKMIPPQQNQXXRVITWQEWGZMORCGMJYVZSZOVEHUDFQFBIXDPQPFVAWDNAGLMRZQCGMUGRK",362,],["GQGQDIICPVDPAAHLVAZHVUTJAKWMMTYAPOKNKRCFAHPRFLTJQNRBBJAKVUMVSYBJLZTXWSGGJWJHTLACOVMEXSEZSSDYKWBCXRJTJRPFOMFIOVBVTFUQKMWSOAESRHYGFFRJLSXDWYDCRGNYQVSNRLZGVCRGQVEYVOMMJRVFHRZCUCKBGCWLRXZQBGZYAYCCWCTUGRGWGUZQDDHOZHVDFXCEGGRXYHWJIFKKRRYYRJTRQTKCISEWCSHQDMRVNUVKRUYOXBYPQRLAPECTSPCBKSAOOZMJMASTJPZVUOMSCCTSZEEMX",932,],["QWWLQISAWKPTURSCEBGZVMOCIVGBDZNBYDVWQUPICYYQAEVSTCOKHZORDRAWZFFVPTYFEWHMWVOTXZLJBFPRKDLCNUFWTZMLMLFBGWJIZWAYGRFZEJADYRPWDISVVCIJWHRVZZMUFSNAOFYUTABJTZLZLARTQMCDCNJVKOLYMVEUSSSERSFONJAUJCJSLDLMSRVZFVPIBCMWRNOLOEKXLCDBSVFGEYRMTNJIWUFVHSDZWRIVCIFWFNKCUYLUDCIZYYEFTIMXDULWRUYVDEUNGLWZYACCDGBJPLXKEKEOKIWNFKKHCATXODNJOQDGMEFPWDRVRLZQSTTSJLRMMKWBTKAYORZDBLHFAACMNPIZUHUIYYRMDLYTOOCPRICYGAUHGDKOQOQBKHUBYBUKMZKKKFFVDAYYLQWLPTHXQOWKZYCPCLPAXSTNLANCAEZAWPJGVNVIUXYBWHBLQTLCODGHIRGNNRMVMZBLPLJZVYKSQQUQZETIGPCUINAUJWEHEIQRURQFJNGRJWFPVPZOIFKTUROPHYDBWYFJUFIELNRPWRWVPXYQLKJNUFQSEIAQCLJAJGXIKAPVOJMEBAKFKRSTFFRYTNSXDAAJNENJGERCJRRBCJGMAJWRTADIJQHFSLANWHGQWABGXCTAOAKHUGTCPFRVOQSEVNYOIBKFXHZPZKECOTLJBGOAZYNAHIKPMTFRXEPLWPWRFJWSJPZDRRYXVKKZSYQFHKNHVKICIHIFLGMDVXGXPMIHAOIZZRVTMYDROBGQYZCYJCMOVFAFTTWNADTFFPMEPUQEMSXFNBBKWNOSHXROZBAKPNVSHUATXCWTWESGKWEETTHFQSSTDHVNROAHRGNNCQZJCQQNOTWMLKAOXFGRYVCWXUBBYFSZDLWEAJAOFNVLSOHXCCQWTIFWYPVTPPNAQMWGWHHAFMEBLPUOBCAXYOPARVOYMMESXQSIEAITQUNYZKTJEVVODINIQBNNPICEGITT",619,],["FLXJKEPXEGLMXEPAPHJHREUMXMSVSXDCYWPGVNQMJHSNZDUPIJNYVESSLWUJJDTPSSJHOMPJHUEEOOFGCUWLJLBHZINKMPAXLLVTCTJFAZWYFCIPSDSAEOPDJOLYAAJJDSIDLDOOQN",320,],["ASKFWMNWACXRNMSNYWCWKYZAOCUKZLZLKNYHXGQWRIAASIOOXGRLMMMMSGNMKBMBBMQOPIVLGOLYYPVPEXCVSQIIYHHOZOWNTABGKETSOWAMFIQOBMOLEVCJJEXNHLYBQLOPQDWPHHVDHVVRHAAQVEKCTISPBPWWLNGFPFJNDZZRGAMMIHSZWVUPCANJOPGWLBVPOCMEPAHXXDDQPXDKQKNZAHVDVLMBMKBTSMXXGDOCCVGMYOXADVAPJBPVQMJXZIGKPTAOUHKHPPBMGTYGVCNZTNLJBSEHFXUSMCHBNDTYBQCMBGZWCMRSZGBZKDWWMQLVEQFHQKWHNQXLXQYBJXKMHRKKZPROVGHNEMWKBHLYCXIOVOLMUNIBUQBMOVDMORXIQVJPTOUNXOTMAYETKXXCOGVGRUFBQMXMJUAUHAVGOTCWMLCUWLILWWKNKPIJXYCSMWEUCCZVONKTYQZZQEPHRMSRQRSLGHWZQVAPZBQHABEQCNCWZTYZVWSPBNHFYKZUOSVWKGJUYPGBELTCLYBIUGRKFSACGSRGPHVYJGZKHOVFZIQCOVYCLSQMECJXEWOMEDNGBMJQKBZOLQZKHRRADEVZJRAZH",135,],["DGLUBWWVLSBEHDLAGUPJYKVDWPYSTQRSEEPMERPESITAENLZJJTEFFVTMAEKJOMWNPPUTPMAOTDLYSGPLQDKGNDVLIEPDMYXRLMFRXJZHTYVQYFWXYAEFQGXYBFNTDRZMVAENFPOGUTJAOQYSQOGPMPEQAYBPPYRSXCMJRMSGYCAGIVGUBWWAQSHXTUWECDDLLOKHGXCCNIPBDLJVEJBBXUEVAZGRUXMORILMTDTXOCQNOOGRATLBCQTMIKYQFLBSOMVBECRIGUWHCNMQKKWTTGDJHNWPKOEPMVOJTTTBDMHSOTJKTGOOIZEYQQILWCUKPVJEJMHGPLJHKRCVSXXNAJFUGQSFKGFNFMAZBDMADFDWNPNGBAPNGXVQVGYTKSNTNWRDSEYGEBVBWNTVWITZHQSIPIWAXBKAWYQKZHRGFWXNCTHWVRMXQLCGIVQLOITZWGUAOUPZDVYHQYHPLWSERLITMJPGCFWMDYJKWLFKHVKCQLXYMUMAMBONGSYLGTUXLFVTVVUDETYIJZJAIWRCVGOCLYGWGCXRWNXTZKWQMQPEVXPJGDVBMBKYZFNLEWGKQHTADUNKSARSYBGSBCOSXOUAOQELIXIMLPZAFUKLSHUAQDJBZDRLDXCLCYFQCXHGGRSXXNCAQAHLXIWKRRVPBWFPJSYOZJWVECGYN",436,],["PLPRPVBYAKLNXLENXQFAEERPHHJQRUELGRGPULEDIIEZDRXIYHEVZENJMLMKJVMSLZUAUDWPPSFXXZJDJQZYCVXVHYAJFGJAGMTDNYCROWLXUWDLLJQCIRJQBDXSUQYRTAYBNQBLBNINSALOBTRPOGALDXDHKZWBXYDSQDHLKNYNDQSNAEAGDFXWYXOPVSAXFKCZHLDLGTCVLBROMIGDBGBRYEKMVDDEHBRGZKGDSMOEHJXBTEIQNZFJLBNXBGXPGUFLQJTITQMWMXBXSPZKLOXDSFAIFBBJJJCFHPNQRDAGWYXLMYXVBSRNSOLHQJYRTSBBBTYGXEQTJMNLQCUXKHFXYOKELKUTZSZJAFBASUKJLFIXPPLPLLUZNVBAMWNKDNVJOSPESZGGDMIXRTYPXBEGJKMHYNZHHMLMNFHBKSYCUWETYBLOAQQZFLPKNX",448,],["VWLXDTDSWUFIYFWNNBWUMPFLWXCXMDRLPPXJUUZXWIMYPSCEUDXNKWXBPVHJFHNYAWMTSSPFLMXWJGVAWNVZMRTORVFHFKSGNHUJTNEQWOGGPXZSJWJQVRJYVYVSSRHCDVVBOVSUEXPGMYHSIKSWGNBXFHWFYPZBYDXYSSEVADPVAGZQDWQZLYLUAMKTFAFXLXOCPQAXLRSPXJHQXOZJJXAICFZXZKUUYIXPRRHQQUVXSXEBQTXBTWRPKURREYGSEIPZHZFAENKMKKEEMDZKYGYLRALUPPNHUSMYPLYMQDKUGCXKFUKNZACVSUBQWFRJMRNRQVVRDBNBHIICOMPRIVYVGILZWZLNIFAJJTDCKMXLQTHLPGKZTCITIJHYFSZPSVOQROLVIKMTDEGIMPNGCZBAUCLWZNRMWTQVZTCDXOMDYNTPVMFVRSAQRFGBPBXFEDULFEOOTAYNALSKRZANTNAXLXPNZZROUXREJROOYYKSFYNIZUOMCEZSTCFXSFYVHILTIABSEZWNTNBUMPQRUXVYIWOTDALTWBODVZUVADHBDPNPEKMSOEEZBNIAQIVBKHEFAVRTMWUVQTYLUWNDIKSGETJKZRWXPOLIEVBLUUFWSWAKDVASUIOWRZAPGQUTMTMCDIMAALZTXTKZLZXJQIBGZDMVHNKDTMQLURABKBBVSFPKBYMNDGSMBUIAQQPEWICBHOOROUDYHKQCQIHDJRQGUYVTWGPLSSRMASRFDJYSEUZOEPMYXYTEBPMNJONEFMRHMSBHRQKCLLBHJRRVQPUKWMBXKUODAHWXGMIDYSCNIKUTIUKHXMFMEQICVWATGVLGNIAYXTDUMSOAHYAXLGFNTBXPHATKEBESZJQODPKACHMCNZDJBWHISNZRBAFXMXOIVZMUWFSTBRVAZBLDFKNJL",915,],["FJLGOJKCZZCUNOIINXMZQUDZZFVFTZYBCITFTDCDPEDFNZOVBBJDUUFREUXOOYMIEQULGOVCARBEGYUDJJDUUEAAQCOXGWPSKFBPLMFPUJCLHWJNKDSJBRDCFYUHWCSHGBXYYJTPIUQHQADRTCMUKJJSDZUVMFQKHZFEKOCFGWMCXGYMEIFFEUCTMQDSEJHBCKKKICEWZLLDNRSAWDMVAWOYTRRONIVROABH",413,],["YFLCZHZQYIBANEYPUNFLYRVIFKWZJIOTXJQWRUMLMCSYEIFRYHUIQUYKPCVZPACWWSYLWSFIYPVQCQKRFXCLCOACRYUFXRHSQTFSVXSIOVPXQFFUWOFJTFRQLQYUXFNDKEXXXWHIBCCMPIGXURXGRGRAXVANLVWGOEXAYQGASMFZKVNNABXKQNAAZQMTVWMBQQDSNJXCIZFYXXRNXKKDWCIGTLSDHYJZOHLXGJTBIWGKMQSPRKDKPKZPGRSRFRZEYMUKDRUTRRJSQGWZLABNJDQGUUSIFFUCWITKCXDTGNKKCWRBEFHUOYBYINJRWIPUXPIANDYVDDEYEKSPIDUBSJILBUUBHSKXXXWTTEMOQYURZJNKVOBTAHAPTZWWDXIGFVPDNPRWRKLGTNKTGKSVZAJBQDVUSFWWITNHMWZGEQBCHEQJIUVLDAAOCAZUAXEGFXTJOZYZRSYQXDCISSTBVMLFUIUWXIGMLADNQQJYMLCYNNTQLOZLQSRSCUABAEXJCTKJDIUMAHHRKPROMDRWEHVITJTCEBBEKBWAGFBSIKOXQAUSXLAWZNGSMXBHKQXIRXQINHIXIMGSGKYBKHWQMVEBLRLWAWLGWLYPCNAQRZNKWSEPCNXPUULXUKBGDYXLICQCIFCWXICPLRWGLBSOYDRBFJKMWLQWIHBGGJFNVQJMDHANNJSYNMHLLAZSXRNEUHEIDTHWJRJNIMENDKGZPOXXHONHDITJSKKGHAXOLKHJMYJZDXLTOCQWYIWADVABWXBLVQYUPYUFFWSSTMWLDOWQNCXYRYOVWFTYXGMVPJZPGMDTOMAUKDNZXAPRNEMJOZGPJDKSUORSANJVTHJPBZJKMHMDNTBWBIWROEGKHLDJB",626,],["JOVIIJKNHFBMTPQJUJJNOTJZYVBPIEIEZIDHXBXUBWRBGKJWQHUVBRVRIWXXIUEJNBDOHNLFMUXXHANHTHNDWZZXJKCULPJUQJOOODSJNPOWIMYHZXWGMGEFKJSCYSXQHHBMLTFGMSPGWYUZGQGMPVBYJEBJCXAQRXXWUIMBDJCCDNWIECWNQCTHSFJHTFKPNXAVESKBSFNUMBYCMXHTGZWCXSUEJDKEIUWRYWJKXRBROGFJRNZIYBJFLIZBMBDIDWYGOHZCLJWLKLFPNCBNSHXBKLLWUBWBISLVEFHUGVYJHQMXPSFJWAGBBFWHJNQXOYMHIAPWDHQVNEZGZIXOAYSJWYTLB",243,],["YXYAIHZCIMXBYWEGILCYZDAAIOBOEPLWILYHHKJYWEJAWVCMTOBMZXGAKDBDDOIMGKABXRIZDAEFDMZQOWOKJNPOBDOFQFOFXBYWQBXWBSOERVWBMALAHELTJFCPQLRWQQWDONRTQSKPQIKYRFAYFWOQHDPIBAWVUVSOGGFZEYAQBTEVZAFTISVNQGRWEPBHZTNRJZDKHQONEGOVQROZIYYQJBLKOCTOUYJROLQVMNLMKNVIUFHJZHONGPVSWCDAFMPSCGHNTAOOSWOAVAIUJSDQQBZOJBVTJLDBLBAHHAKCIWTGWRBRKHOFBFMRJZNKDFBCOGYHVZZPYMSRLJVTXNMTEXCBUFKVIUGAQVTKBHSFGMSLGFTQZQXKBACBWKHGKKYROMQSOOQNJPANOCYLAQDRTKBDPVVYQYUOYOQLVODMXKPGIRCYXOOMQQWAJWAYTYYLQIFHJJIYALQDZEHWYYAPURMYVRSSVESBFQNOMYWOOJTQAWSHUCFARIHBTGDOYZVGJBALSLYSXXBWNAONUJMHSUVYEVIMWOSJHBMUGDCRKUQEIABQQHAMEMKFHJPGVZXFCYXLJYCMUKJOHZCSCXHQHNMHQYENUSBKFJADTFTJHJFUMWWUBECJKMBDIPHUWUWSWMRNMKTOJCTUIJYFAOOLQQJXYEKFVSBQPXKYPXJSIKYXPQFXGIZTSGQIPBQHMTYZCVBYTPJDUOYEUFPHINFPKMCJZHLWCHDPDBOWVIJEVPCXAGMDEHQBCDOEIMKXCUHFEIOYHZTEFMLAACPGWYTVMOCQZCZGTXOWIYXXJPBRRXKOJWEPCOMDMVOFLTMFHEAVRT",374,],["GGFWXIGKZEWTAWVYWIGGKDJIKUWVSUKDYZARURFRJXXGDKVXTWJDNDVXVGRYNGZHPRMJCUHFXHLUNXJOHXEGYGHRLPSQGYFOBZGVGCQIGYGFJZFXJJUVGMVQQPZRBCVLWQMQVJGVWRRXZXFHVTJCUEUCRGEFLHGLUFQHCTSXIWOYINMDCGHQTCPXXFBUPKYEDLONMXNVULFWXRBTMUMDDGNVWOLZLGPORSDRW",853,],["CCRWRLLSMHWLSFTLAAHCIGOYNLDXSWOTIPCPKKHKMDKSFJRTPILCTLKLRKPVGFNXSGLRDZZAFRPHLVFXKSBMOIMEPACBELLSGQZUKLKEFEMLNMDRBJMZWSRJEOTAJNPUMTSTBCGQWQCLIZJWDKBWXSCEFYEIMOJBPSFFFVFFLJGJCUPQOPTTPRVEOWRTAIFTYZCQVGAKHYJRLKWUPWIVTMFOGEPGOPGIZMMCXEISTQTPOXVVKSCHGTLIMECITZKCJKVFTXSGEASNCGYFSXRRAVPXOYMMVEGXSKYWDSAXJYCQNEPQNSNWTGDXKDKVHMVSKAEZPZSLACXKIJYGISRSKAKTDZXMGUAPCSZAHLCLVPEBMKLRMESBWDLMMFMDLWDRQPVOJMSPBSBTGMWKYBKCJJJLXGZSCYFWZEKKCVYREOVDKDVWHTFOVORSEGQDHYSOIORVAIMIZZFGBIPAUFTBSJVUCQOBRXMCGXTEDZHBLLIKQQAVYJXDRQCPEDDQDMHUESBCZYYDDVRHPEGLLQIRPTOVMIMVAEBZNDBEATQCKQPMJFOMVOLLZCUUKAQVMEXVJZKVTKABHSVZUOJBZDZDACDSTFQYPILFOYCTPMPWXC",135,],["RCFYCYXBJUFGDKFXXPVXZXTCLWOEXYLIDWGYBELPXEODYBWDKEKFGBXZQQYAEQWBNBZJODQDVFFYSGACRLJSIBBYLHVBNWGTOREABSYBEGHXSJHOIQNAZGNFKFTOXRDTRCEVXQFJIGPHBGYLNTIPRPLZATATYXAFKAGIYSAKBYXNVLFPHZQCAVVTTBVVTWOGGJQGFYSOZQKYNTVGJRVFZWFQKVQYHQMCANXOJLNPZARTTUJRCYXAGBNBPVCQRFGVBMTNFFTCHPDNCKOIYMMJJXDVGVSUSTPCGRFRFHMLOZGDILUSKXKBMRZPRPWNYDXVJJACPHAMJZAVFNIKHTEFMCNBECFORNXQHZUBAFPNBXMNYZOXEMPLOTCTNAUFEDKOKSREITPBRYNAJQRFEITPUQHFZABQGPVGXNNFBZDOQHAGWCBKBMHNFTFJOPHVMLDTBWEZRYAXVFQKTBXLWBHNNGTQBDGTFTQASDWXETBMGGWDDBODZKQETMTHBTAKPERZFRMNNUXBGXTLFDLCJECJHDHAVRIJHFQYUYIFLJPYWPZHJGFAKPARMCGKFFWJASMAFVAFYHXCZNSYPBKGZJAHPNATYXSRVZCTPKQYNGQILMIOUGVGXIEVNLZQTGLRRHRVPQP",824,],["BKBXTSBLINPCRBBAKNDHCQKKILJZFNLMXDNBLTLKZNBBITTLOWKHYGGBKVUOKOOCCKHAHYTCHAFITTZRGIZNTJETPUBOZHRYMZEEILXWPJQMHCHPPRUHLZGNIHRGWTQZROSAIKEMAMUZFLGLTOWJKIKKUNMQ",223,],["SPAENPIYBUYHDVPFPQBYPHHPGUEPUGUJFPXXEBJXISXAYFKTZTNWLLFCFXQDQOOIPAIWXALOISVFERUUZPWZMRZHPHENXUUVPFKGOQYHQMITVKLRMEZOEGPOFYWZKNXMPXMICHOZJDQHHZPSTFVMWDRCYVGBQXRPUPYLBOEIAICBLHWHBBITJQUSODJSNQFGUEDXBYUFERARZWCTZLPMOCBEMJGLGYAHBLMEAEIHWFQDAGADZVOCJNBRUGHZIPRAVNGGXNKRDRCSDEVHDEYRSDRUZYBBJYZPTEOWXIDSEICPLJRISVFRJGWSRIGHGWUGNAGMCZOSGSTRLEEUJXDRLTNCTBDPBASDUDVKPZNXYRNUDWVAPVFGLOHHACAIBQKYBKUUMXVQJLPEXGGSDTPGAHVBHBRQKBDHPUGCDMGUXATIFJBYDIUIBEGIMEZRFUKIFUZIJDPAQQIYTEUSDLRURTPKGFOMEEGGQMAGXVEYRYWYMAUTSUWGMTAOUYAOCBQPGJPULJEVONJZJRFGKKVUXWPZKOTIHPXHPCYHJBUWGXTGYECITKTAZNNJIEWQNVIQMUIOSRTHYWETOQIPXEGTUTLUHBYWERAYJXVDOYHTUTTXDDRCRJIZZRFXIGTBBOJRXWWFDBCTKCLRNOCGYGAEOECDGGUZBJBVZNWDGUJFZKNKLYYYEIGKFNEXTPFD",525,],["NNUXNCDNGGNOGNDRLNEHDLLXJOHBNJDWVTFNAAJWXKXWWWWNXXNBVQNLDWVZGXBGVBOPCCFNYOJMLQNCRZPNQPQNNFEDBPJXHMQIVORJSZQEQTSIZFGPVUZWJGHCUDEEAKXOBLPJCFAQVCVORGGWNKWONIQXVZIENCGKOPKTYWBFPCHNXDSTVYTRQAHILEHNHWGVGQHXOCMBCVOXBAKSZCTESQGKCTJAISYKUSKDESUPSCQTSCLWYGPAZQNTBGZSYWMDFCDWZCHFQFZGLPQXBGBZBXNYUPJFYTTHGAMAVNJGCKQRELAIVCUVHELKJNPGVUKQCCVBTAVAHIPCAVHXCYVBQAQABEKPBPEVGBRNPUYEAFGNUSFGQNBUWFTOQLCVTSUIZTIFRZXFXIJXTRUIXXVCZEQSEGLJFSYDEQHONYGMSISUZJIDTNNSALDZFNQNMGHDIYRRWIXKMROPEMBVYNHPOFJGGEPJUNKGBEWHQNPXZLVRDXAKZPKRMNHDUPVESGKSAFZVLPCYHVFBAGLPXRVXLMRNGVAXISIBRLPDSPZDBREXSGGCVSSZXTFJQWWVMQUARJIQDSJJVLAHHDQODFVGIFELFRWYNFLHURTQSPNELONWOJEWZXNUVMXUFVDYKRNFWIHEZIPKPJPKHCZDJXHEGHAJRMDOFGHRRRWARHJLCAYHAACVIVHIPCKKNTEHXVHZUENPYHLJXYZGDUYYZUFJHTNORVJXLMEWNQBCXXVHHSA",13,],["IIRVWEEKHFAYIOWTUYUOPESUFIBBIUTIYFUXZUNJPZDUOPPDOUOPYGBIDVKBUTWMUWNITKGKEXKIKZALRLZAZTIZRJXBKOTBPJKXRIRHLEBFVUPDPTEKLPGLXLHISHTZPIYELFMZEKUUDBMAMQYAIBRDEQHOESYURBXSIRPICLKLOYEALFJWWEYMWLIQMUZFSIZWYVJYEGHKMMYODIAEMPSOSSLVWDTDZBCAXPIEOIKDQIYXXLXZLSCJRKPTEEDUABQAHESXHUGVIECGMTQDSOVVYNSGXBQFXEPOOVSKPZFWIKUGOXEEDLIFGKNLVGNPJCOBZGXZSFAYEHBECENCXQGNGHEXDVKZDUKIUXALVPEMPUBDXZMGPUMTTEXIUVQKBHDZENFNTUELLWAUIXSHEMRUCWETKDNUJWWVMBXNDLGLXAWVJHAPWOHVYKMLBQAHBJXYHKJYOYDLQYUTEQGUJRZWASUZFMQQYTTUDLLZKACJLPLVOVFXEVOYWXTTPUFGDZXZBLKONCDVKJJBFTBUJHKZQHRHWACFCUQCMJTOQYLPLDTNMWDBNOTRYHKHFQWKDTLUDWHEMUOGCKMXWDWSHRJWPDLRIVSPHYPRQIKZTIXKGZVEYPPVXFVSDMCABKARNLMPXRQJXNJUQMMBPJVJHJGXNBMNFXZUUAMQTIUJNBRHBNFNCKKRUYFLPPAKZUDBEFQDSRKUPFMFAVPYTYESCLSLUHOFOOYHEAXBDHJP",903,],["OXXGZXHCAFXYITJHGRGYCOJNBZGUJHXLKNDJXIXZPOSAFVDXXINAZXHYVBDICGCDMQPCPQBGBCQGRMQQTZNYEJAKTBCQGGRBZGNOBBERDPEHZGPGISNCXLOGEEFDWXQAJCJDSEUYCRVGQTOZADWZIFKTKBQFRRUSEISPBRUEBSNJIEUQPEIXCMROQUNTZIFHYXGBFQRYFSVOOJXXIDDCLHGPNJGYDHXUCQKFUUDQDMWPKBJMONGSYACFJIHBWTHZNRDPAJNJFJYRSTOCEFJDBZFDGGVGFSDNPIOTGHESDOSYFHXUMGNGGIBYRMHZGILMECEFDGCPXFRRWBJZMXHHROPDQYUMECDQXCVRGWGGOWACOJRZXHNAPLPQJRVEMXIJGDJMEITDEBMGTTVJLNUYHJYLVQRWJQLGPYCGCYIRXCKSMHTGKNWRYOEXDKVXDTLHBOCXJAMDQREEQPMYQJNNGYGAFCPJPTYVFFLDNVQRWRZTCXGKNKWLJOTYVXLJJOXVFXHTQEUKFINZOFWYILORWJLWNERZPRGVZUXJM",15,],["JAJSDJFKDQCVCADLAKTNHZWUPADUTANACDOCLBQLZRJFXVFRFQXNRHNSPTKSEKXODLNCFSJQTCPYAXHXROBKULXYXFDXPYLQSHZDEKIWHMJXMOAPAVDNDNENLTBKRYPVNBNFNBTXMHVEMRSHZJSNZBIPVSQNGHGGTJAXDTMXSHCCMZSENRQODBWZUDLATEGCHRCAKKTIWOLKRLFHWXFSSLVJNXYVQQDKXEWWBOFQHTSCDEEBERSIKUAXFVIJUZTEJLSZLTOAAEDIUEZAMYUEBICKGUITODLUHZJTOWGWWOXFVQFATAXQBQUDLVQFDMSXDIIBSCXFNMNRRHZZUAAQRVOIFASMKRARTQHSSCTZCNVNDPDDKECFXHGMZHOSMFZETINEFNWFVZZAXTAZTKFAUBRSUDXOWRBEAGYJZEDUCAUQFIACURMUUSJHZNEQVUAIGHVHRQIJBFSIDQFPPNLVNJREZAPCDXGZSZKNNEPVDZPLGFRSDCFBKFJNNFOIJJDUPGDGHVOXTGBSIHKFLFKVYFLSMSMEEUYPZWFRCXXFITEJTZKBVNFMIHBAVQJBWDLXGWTQOQOLIZALUSSIXBACYOOGEMIZCJZDEEMLDJVTNTGQXKXSAWYBHRGQLYLQNRIIJUGDDAYBTHPIRBMLUVOUEZQRGXZRAAORNTHMSEXBMKYTTKCXPRDPRTVAOBAEGLRROFYOKSGFJSMAJWFXNJSSQNFTCNARLYZVUQVFFEIYSHOVABWHYMSULZRFFFNDGPFUOJZTNMWQORGSCYZPTJOHBSJBYJOPPEXTURIPYTOSEPMZCSIXAOZRUEENNTHTFFGVTNTIIRKAMXPTLYSSTDYFEKHKDWQGVOWGTTWNBNNGLJHLCPG",166,],["FDDMFNFHIMPIUFSCFVLFJLDEAFVZDRNSIBLIFUHXMTDNHBBSBEFSNDAMVCDIALFOZBHSPGNBJIFWYQPDDFUDGNMAXEUOYOUKOFCUSUHBUHBQCGIVYRZZEUPWHHVTTXLCBCLDMXQUZUIENLOJOAWVCDGYLXEKFDSMCQGKMHUGGDEEQNNAFGSPIGUCFRNXQJZMZNJZFRQTTDXIQBQMVPPEXLDBKXOJRJQXYVSAIVBYEHTVZSUPEUIMFPSPEWZADLKZPWUVNMRGSOFFKOGHGFITTZKGGQHGCYFTIVKXWVYCSBMZQESCJFWGFAFTDADHDTVCREMUZBFFFUBFXDNHJVEDKCJNSGDKERGSNNCROQMIDJRYTBFIXZHYMCRQTFCHNXAIYUMKTMECJBETWOMUKPUPHHOZDGNXBUOFKRKOZUDJNOHKSKHFXYGDVGDWGGBXFQFIAYQBGPYQRMDJAQTEPWXHEHQDQQZNFYDGTNDQCHZGLOWYJZXFSAZHPMRWUBPOCCJDDNRVXUMTPXNDBJPORMHUBUHHFFGJVVAODINVNKZRLPNNNPRVALAZIKMQOXLWLTQKKTSXQNVWHUHFPABNBGEHCZRFXZCDGOPFIEEADECEVGYSFZBWIUKTCOZJDJTLPZJNFUFPOCFDZDDUBQKMPWCZZHAATIVCWGSPSYEANSUPPNOBLYHVHUUFAQE",698,],["MMQZIBJNFNBCFXIKMNSMJOEAXMUMMHKQSNMZRPWUOSCCQCAMOCNNUHTBABKUGJHREGGYBNTADAHHXFXSNCQRIQSJJLYOSRJCJKDYMVLKSYUNJMQNWTDJUXFIBBGSGRMGONGZNUBHSEQSDCPDXWBTJZPUJMCKVOKXTKOZEGSRAYSYYQJMTBGNTGEHHQZTLGLBRCSBJTNJANILPOSKUQPZNOZKONCBDKNXXIICGNAQXTFTBSXNSDVTAWAKKGSNQLSDWNNVCHIOBSOKFBLCSMKHDVVUPCVAGGIIEMEQCFQOBPWWIRCDQNJXDIADLSLXULNJJOJAUHPTXXXIEBNGWJTCFUZHXFRRUVHVAAKX",222,],["UVJUZKHKHDQMRSKWYTQKWONEOFYVCMAUKKYPAAKALZYHWNKIVNGCQRUUPXNYFQABOSJYUIVZXGKOOOLCJPWBEANBXYRGNCIBAOEWPMUDOJENZHAIWQIDNBZAGEDJZSEFVYSKINSKTBAMTJPMAMZSFDJRYPODWMUZDBTEEZNFPHVFQOJGWUZQOWHJAODMUFOAQRVAIFOWQGKRWFEUZMXTKEFCKMHYUAYBJEODRIHEVLHBTCVOBGKELLTFAGPTRMIJDRFNYRDCYOBJEIDCAABJKIIZDYXBSBAPGOSRNTRIRTAYFIRJWQAGFOOGGIRTBMWMIKOSUQHBHAXWAFBHMWZVOZLLIBEXTPATKURPIATFALLIZLTONFETNWLZRFOUQOPPVWACEPUIKHYYWOITMMGVCPRZVLWFDLEFLEZWFKLNYAXHFCJGYRBQYNHYOSVKAPHPYAKBOMSGXOAAAAUCMFCXHNKVTYHHZIVFAMVYSZYGFAYEWAUSLABXPIIJWTOZJSJ",68,],["QQZQJMQNQIJZQNQATPALXQPQNSNHJQQWANALEJQVLNVFHPNCTMKFSYYXHAHANVWQNVYNGXGAARWBIUAHNWLCILRNFYYQIQIXQQBNGBHSALJXAJZIGRHGYUMFJLYZIJBURMHYVRTGRNYBNUAIMQNSJFQOHMAQYEHAPNOYFCNUAOYJAMJWWTMATOVNDEFWIXYEMLIUZBIJQQMCWNENRCINPIVUBWGISIYRASQJWYLWUKROLBESUBYNYJCYFTLZBLADQMCJJEAMZBPAQUJNCOWMYTVAYWACXBVBVMIYAKAINFOTULSQRNQUKZCFLGITIQQMIOVXCWOPMPTSZYTSQTMYPGCCXYRPHIFBRNTMRIGTXBEBMHVECDIKXPGTNHCZSRYLOIKZFWGYEWAMCABABIAJISMGWECVAHGCRUOPKRUDFQNKRPSJOSHKMNPZVSKAMSCZVIHSGQIDHRQEUSFZQZLEJBHRCNLZMHECUWKRLORUHQIONPKVHGCXWIVYJQZYYECAHGUQTGEYWWRISURJFHNZNJGRJGUXQSQDFYTSTROAYRBNJLWZZPHZWFENMRYJIUJZXODLDMJOTHSRQTGKWSEPRUMGQADBGUHFQDMGPOMROUGUTPULRQOQFSRLTYSJZLKDGOWKXEUJGUUKQMDRHHITYDIYUKYZGRWEDBKGNCGHINTPOBGUONSSRDBBJZKVQLOKWTUUYLATKKZSTEIGBLYGZKNTTOJJWHOEJULUMTZNKGDQFOGTRGVXTHOJGARFLGKUXSYSTBKMWGHUJVAGYFXLWBQLMMZBZLPCCYQEMJTUQVSIPOIODVICJHOYXBRKSMDLZUMWGKUVFEZTPVANHQUOGQCLTAVXWGAHDJHVTRLRVJ",30,],["JMDKUSLPMAZIRKHYZNJMILTDVRLOYMTBTAMSBBJHBIKQMWLQBYZMYMEPWRTPJZENMFIQMLVJXHXFVWREWSPETRKXHAKMRDVNEJZRXVRLTDORWOTBYXDNCYORZVAXEJUHZSDPBLMSMMEAUSNFIVVHMPTFKGQWNNXEPQITUWGTDFPZZLGSLFDMCYYBKDMQLRBJALEANOVBVCRMJOXRGYWEJEFRXXNUKQXGTOXEZKFNWJBSFFNOJNJGMYSMAWKYLLNSSLUJAWUGBNFSXMLPYBMUIPKBYHFFBDFHOAMBDDCAUVTTDEUWGSSAQZEZFUJPYEGACPPZDTMTNXYBBYENBPTMDKWKHJGLOLQWDHDGEAISGZEPKTRMKGJXZDZGOHFQAFJHRIFBONZFUGPBDQMZFDCHTMYQPVBATKBXVUZSHTKXILMQSGPIAGXEDGOAYGPUFUMEHFWISRMRBIRSIGJUOIIGCEZHYHFGJDQEKYWYZOUCEAQJGTLVLKWSXJUEPCKAEHITYFJBNKWYDRFUBHGUYGYPBOZGSGMWFVEVVSCILPDTMBWNWYGKJFZLXFGXXWRIYJBGESJMXHVZTFETRCQMOTUZFMHMZCIIGEDTXEBWGSGHJUQDQCMGXKGGPCMCGVNXWXLZJGABCBJTORWXSVWRAJILOSZIXPBRGTLGPRAPHIDEEUXPGMQ",947,],["LWNSNESKKWIKHHSJNOOWEKEZZSDNXZCLFPYWBWVWAGSRGAZXPQVGSEYSEGYRDOMEJPFGNWHADEDLLHQHXBODBFFNRLMFRJAYPOVRPGBBYSYNZJCDQZWIRDWYAPZCIMHMSZXZXAVYQAPCLIJVSSWVRZESQLKZFDKOZPAEXGZCKBHZYWATSQPURWUZILQFPCLFPKZCXNVLFPETPFRNFVBAYPYNFOSGZYIQHOPVWHBCAOZPSBTJZDRTNZYZNSFKOGXBBDWWIQOXHZYWZMTYLFNAQQQBVETVYKOTANELEOPJBZWSCOROHEHHRITCKVJVKAMJOVWNOOXQZFHBUJRIZVMPNMXYLIHNPNHKXSJYDVLNQFBAMXQBEJXNGRVSNMJRLYPJFDAQDJWZFAGVQAYDCZVEMGWRVCUCPEDVAROLNPYOLJRNTDENHBTWPGKFWMNWMDUEWLZWISDHSKVXAPKSPEGBDTQLUNIOPFKOAQOFSGYNGHFGKFTVPYSELYWFWILZOCYPNILBBJKQMGJEVDTNFHUFIGWLVBTDFBWZLLESOTR",996,],["FZLJKWFIWOVJZZIHZWZVJJJUZIEDIZLZRLSEKABLXFVEYVMBPOVKCBDWSZMQHQIBDLGIWNVEQWBVEJWNWMKBCDONTSEYHNHHDAEYBNIQHSODJXWEBKGTEOHKAZMZTDYLAQCKWGOMWZ",630,],["ACCMMEOMPEOPCMORTSICERCQCZEEALBZILQGOBOSZBJUBUMITUMQOZCSNBMRQOLTZQLSTZSOOGJHFZZKXVBTLRAFNHPOOPBNGDJXSIHNHTBNJHMFIALKFCRZIHDVRHYMVEIYNEWVFZNUPRRMSHRVMEYEXVPNJRMRCNBPRWTMCKECIQHSYAMQEHOJTWMCNKBYXUMVWDDUGCCAKOXTVMDBLUQLPAWBMJBNOBJVSLJBKXZIUFZMNQHVIJYQXYPGHOBRGORAWBNTPBOFJNKBBTNBUUNHYHXGSVBKQRQDMUZDXUCLSPIKOJFJAUPRVBIGHSSUKDGBJTYCMHOBFUKBRBRZAGJTGQCUOHICTBTXSMEIIUOAGMFDWJSHYBQLILHGDRPYPHKWLUJBHJCUQMXNHWBWMKQEXKOHDMLOIRUYFPTTACCJYRTFPPJZJBAEBLIBFRFYHDLRSMZWUVZGIAZMTWEBYOOTGBXEEOFWQIVLHDOCQQGHQPMBBGFONLITXLHDAARZLKMRGVLZUZZQPNAIURPAIFOEHSCHSKHSOIRTPMHFQBLMGIKMURLDAXZDATOJNBGZFZLZJZWTZGRUXRORXIXITCYBTQGVLNTBVRHENARDZZOISDYVGGHNIIOVJZREFTSUTZNRYLZWOOEZTPUZZKBLIFNEEYTAOVETNSHLSSZHXOKBROXKCABHLHKAOHUTUMAYTJQXIBGOFVJVBGCWXLLGNGOVNPYBACPUEDCFGYKCKSNRGKBKBLAMDOXTVKEABJSIVWVEWMFBDCWLUGVKOFLOOHCGRYUMCNYBFRYDLSNULLRIULMZGMFBBVMCJONWGFFCCQJXJEXDIVNLSCBAYEBVYLZBLMSNWWXABFMKAWVWBCHQVLBKYQDFBKOVJCYCWIXHSOPASYXJFBOVVGSDDDIGKQJVZCOLUMDFTWRWUOLZNKQJQGKEEIZUOPSMRGFPFRQIWXL",588,],["ZZDHLZLBZHKSFFHHNGHDDHLGYGZSSLCWLUPDXJKGHZEAPTDHGVHFDCZPDKVKZKNAOOQVFMCRUSQQQZUVZPSDDXGLIJPNGZIKSRXIKWGIAGCVJHTKNWXSKPVKOEKJZWIKDWFPAXHINSYSYQKKRGKEAEQHMOUFUPTSEMPRPWEVXOCDHWZXYGTRQQRXRMGCKCBVJGJGXXIXDJQZQPHTOYEJQDJOOTWYSUHBZLQQVKNDGMKUIMZLRBA",920,],["NBSITCAHTXFKUXEMMLXODHUFWURLEDZHPBHBXNHFGRGKHBDNGDQJLSTGKAULILOBSDVFHOLBEMLLNINRSUQJEHWDAXGONEXPXHULDVDFDRYHWMSKYTTNIQFVXHTOGMUXERHRJBPRFCUUWCAYBLFLYEVACEDPSRVSHASJSIAIELWLPAHPGEMQADHPIYRXZNZPPMLBWBTUEAHOZBLGEXMXUDFGGAPOSFRYMVHUWSHUIMVYGLDVGWELKNBSJYJKDBXAVXESRQECHXAJQSUNSUSEEWATDBWNXVZEIQBLILNTGEBIKRCSUEZLXLRAPSWVFFXDFWKZDWEZFWPPZAGLUEHHEZAVTVFDC",206,],["CYIZSCSHSMGCMXRLDOCODSSRGJXASZYUARXWZWZHQXFNDZZJKSAKTBROTGIZRAKPDQNVAKVMYJSDSZLLALPWMOXYFFLGWFLLPMDCKSRQZISGKAMLIVLOAAFCIFFNMYUAZEYUXCBLUVCSIWRQZMVZYEDAHIJATXGUFQWUDNFKUYLUZXSIQBQAQDKCAZJZCATMLGDRTFHFZUMXSXEXOSSUMKQNGQAWZYUECANUWITXOPXERZMAZIFNICUYINBWCZZNVRYZZHOIRKYNFIRULGNYIXAPUOMDWMFIAKJAQDUVXABMIOZYJXTDHXXJTKRZTEGUIUYPJEQQAEQFUKVWECZPUOAODYQXRXMAIYKYPYEVDLCZNSHZZKYOWZBISIFFTZGXKSGIXOQUOTQBTROTGJCQVSQIIYLHSTZUMZMBLFFKDQSNGADKZSNRKM",120,],["FVSSFVALQBZROQPFJKZWQOEJEJEJVIVZBOSZCICRXVNMFDKRBUIATEXWBWRCCHRLCTLJOLDCMTKASECJBKGZIDORLBIOUXQRSKXHZWTCPULBOLVLOBSGGROQTBIBISVPVSEPJGOTNSCDTSDGDXHUTYZVLAVJSVKCCHJUIMMMNFSUMQFYWDXYPYIOATOKSNLASSKUMRPHWHACOBTWIXWESNIOCSQMPCGXSMXHFMUXIXFZSCTREERHWSRJDKDMKBATCSQQBSGWJTLUVKVRJVRCACCSEJPMTJTOEFPYTHKNMSYMBRXNHVORKPCPPMJMEELYBQRYCSJYWIDTTYNEXTPVMZVBODGZBTFBOUTKSSMINRFJSJKISPZSOHRVXUVGLSMFQNZUSPIFFKCOEKRBYZYKONUXNWLOHDKTPPMBTCPGBDOLMXYJMJUNUGASINFJWXTVLUPMZLQNTRPRBPGEHMFUJCKEFRIIVUMDVGRVQYVEBSTYGEBKAODYMGDXCKYFMSRRCMTJTYQIBFPNBZDWSCUNEKUITVNBAUINTHFJUWTYMHEGWEUSBDDIHTVZHWJSEHPBUJTALIQXLIEETHEUVEYUCPYVRVHTJCNGADYKIGKQMAVQWDQDAIOPQBOLCUELBBRRSZXPHSSXIIZCEPSIMLUUIQSYVCUCKMJQGHVUMITYCMHWNTBTCECAKASOFUUQMQFPVBFDFNJDRXBIDARCHFTPJYUEEBBSYFLEGNYHCRWOLBUMKSROMPRDAWOYKYMCDISXSDPKBLKAAHJQZBTWOXYGWOOHRJTBMIDXGUREIRERFHSALJJLIRIQTCUUHBKSYSXZRMNXTQYDWPMDTJLUOSSLDQLEADKFLUZCNXKHQMSJWJEROPNXDEFOCOBPZBILUBKSEUWLMBYSXDWFRBETSZPVMTMUMZFOJZREEOXBTGRZJZMYLV",603,],["JPRAMGATSJAAJWGIZQWOKSDDQWQQJCMZXEISOJKJVDDMTAASMGUSPSKWAGARCNOPLRCUALRGDJFRPPOOADPDUJJDCSMRYPGQIKIJWEUIOWVCSSNDYANWZDXIWUKWLIAMAPGTYMIEMVPRDZNWBWBISQCPPQHHRQBWGGPEZCPIECELZTEBINGEBOCZTKGXTKWDPHMSIVIHQDZSCMWGALMKJGHHNPJCGCTGEPIQCTDLVHKKOXLDPCDMAJSPDSZLBZPRUJMJAUTGNDOPVNOTLJUGOVMGUYQYMNFHRAEPFUBWJBBKAHOLBKQJDZIGQCCKILEWURJHELPRSKLDXYQPQPPKSFTHQQVVPOEWRVFUWOEZYORWNJMZMFHEAILRJGVXCCQVBSIMAXJPSOWZBVVJOOVZIRJRDKAZQLTPQHKDXXRLDZZMHRLMGFMTCINJUHYHTNFSCTQZAOSLHEGRCYWZBXTMIELPHLSPLNVPGPYRHIUMVRVEFTJBAGYAVTAQDFHDJGBUVRHUCTOPHPLSXYPICJVXFERRKPPFCNUBBXZZHXPBEGSULLBWPANUXGEBHLJSXYBLNIPUFQEKGQEZFPWWHAEDQFNBBCLTNYUSMPUOGIRDYDVIRGMUHSVODISFYUKMBJNUYPNAVMSTVBSVHYVVRGSFAJGDNALHIZZLBSFBSBFENHMOPIFWLWRDSNULYBRNLNTTSLFFBWBJPMMZEEDSFBLAIWLAJWIFDGQQOKCHQVOOSNTQ",226,],["IIPPILYRAWUITBWPKVGCPZCGLAWUFLSASONGMTYSADIUODPMZYKAZVNQUVAANPMTWXDXULJGKCHYZRWNADGTODULDCAWAZKKCKFNASLMPSQKFQUYIGTTMNZMHZMYOAUODZICTJIIDMCCVKXYQCCKNMDLOBJMZRWTADKOAYOCKOGIPABICIHKUILAFXCMFEPIQCICRRYHPQDIWAVAFJIOSXOUBTXSOXNNLBBPADRKEXKEVCAZWCOOUUSOHSFLEUCQTOQRIJWWJLRXTLLYMQPDWORWRYILVXQEQGMISDXLYWANQMGQMYXDCYMSUFKKUHQPEJGCWCYMVICRABSCUTRFBDJZHOBUIBSJONHLHLBLOGYITFYYELZFULV",580,],["UUCCEGEZGEMXQSSUBEXIKEZICCXEFEAFAXCKGAZACXHUQYDGLALRIVWMTMUQDZFRGELKWGUZGVCIXWURPSRTSIODZGXONIXUNLFTKCSHVTQRWQADXFXOPKBNKAKAILNBECWCWQKKMNNVBBWTNTUYDVAPDHHKTGYSPYSNWLONOOZRFHAWQHXLHGISHGXWVGKKLQAVFDZCAOCDAEWWKFIBRJPWVKLTFEAJUODWEKMCXKERHUXXQQEFVRBHLWOATHUGXQRFSNCOWIIJZBTQKZRYLLZSXPMVWGWZDZQSSNZILGXQXZLZWTTPXZKXWLANHWTWEPJIESKFKBMAYVARGWBEVOEAYCNLVKVEJZNVVAHZLNMPOVRIAEHUVBJMNUGQHEYTWJFGUGSJYKBQUJIXDRHOAMCEMFHLFSQXEJRXTQDDUZLOCTHVMWFDLWHEEPZHQVAYIOLNFOSNKBDHILNOVSDVTTIEBOKYBESZIFIKLEJFMEIKGWNERLTSFORLNVSCJSVHFNQXFFGHHAHCNOOYBABPPDUMKQJDLSHVFUAUQYZIBIDIVTVJVNEPMNQRFODZIYTKGTBYEODAESFCKGMLSADOBIDWFZDDMSSAWADAPJKCCSJSINKJBPXJCPVNNGTNCHYCTEJGSRGFGPUIRJIPJWTSONLXGVZFTBSFXBJJKYHJFWORIFAPLPAENIKENETYETKBVUISBTKXKFJIISJCWOTOXDEEOVDLCOBYKOZAMWIPRGZDYHEAYLYTYVVQEWKUNUQQSCCPWEIOLURGJYNMQVQGKAXQIRFYIPPDUCJELTZQYECSRGZDKLHYFJOXNQIFSPETHIBDKGXDKEOBWFLDZCDWCVXGAJHPECQRIIZJNBEALPWODWLXREQYZDUFTLFCOXIIZBFMYVVYWSMYVBJVPVEPVRKX",293,],["WIIWEYITZMKDEERWLKLXANPXUAXDGXKTZJURRNGSJUSKNEMZWRTKBFRMWOTBJONQREZKSXKXECPYGYAJBATWTWMZCTMMFQCDRYQOLJNXPJZKKADRIRMQRRIWZRLLFLTTGFMKNJJEZEVZUFXRLIAHULWGLSBBTNKAYXKVRAHTPHIBWGLSKIMLZNTITXSGJACWWSQGTGSPNLJTNCFWJFSMEXWOUIIFXCDSDFHXAFQVQRKJZKGGXFNQCAZOCTZZWEYLXVNUQBJXTDKQIBSPZZLNMKAWRZZRPCWMKGUQVFZZFGWSSDHYFZWBLPNPQOODXEECYCKNQWLJASQCSAWWQGNOCTHLUUNXBGLJCYQOCIJZNMTJSZKKJSXPUFWEIVMCPUCLGRNWOUUMRLECEFGJOZFUYMZZDKULUYZVKCHTZFVFNSSBCPQKBBBPOQEMXCIMECBTQCBAKTJQGIJPS",950,],["QFPLMRGHFJUIJPJIRJJUQEUUVLECSJPLCJOFJORELGNFJDVERKVKQECWPMOMTNXBPXXXTWHGRJUMRTMKQYCHAQASRVMRIKMXNYCCVEOZZQYJFKRFJVNVHOTEBOJEOLJGPJLKGZNLIQKDDNZGOZUNTWAQEEQAORIZNFFWGFKPJBJVJXIJQXXIBOPLKHUHBLDMBTCGNGFNMZTBWQRBTJIKJSGQVMZIIWKFINJFCUQHJMCPGNQCUCLSVWFNYMGHXIAIVWSYMLZZTIUBLQHTIXKZBDLXQUYEVINKXJARFEYBYJNOGYYVYSLAKWTJWWTWNSUWKEAJPZWERBEWAICLMXJVXLLQFTJTKEXCNGFLIJJEFLAZGWBIAFOCMHDKCAEFYRTUNYEDZSDYVAIJEEZKIHEDVETFLCQFGELGGEPVHHXWLMNNRTHWIZDZTFSGZFGKNCFQNXOEUNGTYJGEWUGQLIATLAYLOVQYCEOLLOQLLOFKXFKTDAVPCWIKGEJZKDUKHIYBTQBQEOJCNNUVBZBCRABNUHQHYUWO",945,],["XYLDIBPYDDUDEBUPFBABVOIBEPACTWPUVYCDDIFNGFSWOFSEYSHQNNDPIHBLIBETYMYWSYDWSPJMKHDBOHTTXOPVHMKHONKWDJWJOHICYIZEJKGZCMVHQWCVCTUJDJIOZDEAFXJFSAWWRQRFOBWHSENUZVQCIWLTGTRYIHDOSPDGPHOSHCCZLBTXNLPYIHFNPXMXSXYIJBEXLWCEZGTIXGLLCOIZKYFPFNWSTTATTHMYJQJPJVWTHPQAGHGJUSPDRAWGRNCDRWAWUWRLNJVWWVFGIDCVGDSCZHBNRHGAGLMAFUAJDEGENHBCXIJASSFKEPJIFYNREIUCOVBUQIPEOTOUEZLWBAPTCMBDVKLCCUFJCQZQCMFBIQIGAXHQJIYGMCESPVZYHXKSAECAHZASPFTKGKRCVRHTIOEMQXPIFPDXMRYYTWQEYEWELWVCUBEWZVKUPRSSSABPZOCHWOEHNVYIXBUXAJENIQEBYRVDVVINMIQIDZOLYNYYPWHJYROCCHZEQDETZUZSXBHVHEUYBXXYDQREGWFFLEKVGYQEQZFCQMCRKTZSEXNUJTMYGPXPPUWHAWDDRUKCXHESJSDOKQPPXSPLLCWUPQFTXLDSMZDYYTQGRNHKMUJQZZVUUTCNQRDIFJLAJDQZCWAZMBFATWVQKDKOQTRSBFGZAYDTNUPBUJQVFZCSCVGXZGEQCVZMAMZABWUMNNIRQUNSHOJGHWAJZPIHYACMJWJAYGCWVRVCEQGXIXVTJXJQSMJIWMVVIEDWMJXRAUVRGGVXLOPRVTXLFLWJOUOWQTWEWDSEJJZWRRBVGDURORYWPZXTCQPCBXLFLNLBDJQFTOERMWGJQYNFNPTATVUXAZSXPQTNNRJBCLRAPWZFLXNVGBDWZUUDNEAAPIVDQESIZARLXCJHZQZYMNLFBSBYJCZRECQBNBFSHUKSONJFFXBABMZIZQSQBJYFPUQQDCNPZQCYCGKGQZVI",858,],["WWGFKLFWWVWBJWFXYRTDDFSKGEQXVNZQRIRBYIMNZGPHOMRFYARNXKIRQAZZGZRBDYEKGGYGACYHZYUMTSLWJHFXEJYYOSZUESJSVKTJUJFGIQVDUVQLXSWBGPNRLYQYXYRZLOILMKKLWZUFYKWLLJLHMLBWMWTVZRASMZWMHOLOZPXXLHYJLLYXUWIGAKWFTGCMVDGKTIYLCJGYFERSAXKSOOJTUQPYJPUEIPFMWTHTILJKEPVUAIYPVSJYJVCATORUNHPCLVBUNOLSCZREKBVPLFPWOZPLRWTVUYMXKLSSAZPIGLRILYKYJTIYZYBHGBAXKAQGPWIKMTBFSQQEEOYJSCLQUKWHZYWWKVOLVXXOSPBMEGWJQTDYMDVEIBHXQYHUTIQUGKVJCUUHADBBYGHQRFCUQMVHGBVENGUAVDYUWXVBLYABYEKJCWGDCBGMYFZPEMBSABNQLRBZPJWYSPDYHITEFNXLJPASDMSCXAFMQFPFMWHBGIFSEKIGXLBCIEWRVITQXFCUUJISWTXVIIGWUEAXEMQRYMSDXWZAIVAWPCERVNYRMLXJDGZUOJNTBHWVPSXWIVAWAIVQLHEYBJSAPHWSUWDTZCNXLAMNGYKTDVJDEQSQXDTGYTKACBZJEAQAWHPQHSNMHSUHGPTUATISABDASTSOCIMBZSCXWFJLKOJRVYBUEYUHGDYMPBUIMFWFLADZXHJEVJHCHBOMQXSVWBBDKMCUCUGIUVTNXKLUSICEZZXKGIIFKMSPCVOEPWMFNLAZKFVYTSWJJFHCFAZDMUPDTMTIXXPJTSPFUNDLVIFIATWMYGZDCDRWQXAESSUKOHPHAR",651,],["OTPUQUVABBQYPKJQIVXFZAWPIFDPYWQJJPUZNKPYHLAQHBPHCJDTRYFZTWFNFRBVOUWSNUGZGUZFXZXLUGHUVMQHURJWCACBQLAVWOJUCMKCQQUHTJPJFQLAOBCGZAGOZTVVHYYQLWMWGJPOOIBQITITVPGHQWUWTWMSLLAVYRPDJWVFHLMQGYDKEIXZTRFUZOMKYYPGULTJHTKCKCPCTSTHZRLKRRCTTDSSKIRMCIEGLNWKOHQTHMIICMRJWVUCOJDHTVY",286,],['ABBBBBBC',2,],]



for i,arr in enumerate(s_arr):
    if obj.characterReplacement(arr[0],arr[1]) != obj_3.characterReplacement(arr[0],arr[1]):
        print(i)


In [52]:
print(s_arr[69])

['JOVIIJKNHFBMTPQJUJJNOTJZYVBPIEIEZIDHXBXUBWRBGKJWQHUVBRVRIWXXIUEJNBDOHNLFMUXXHANHTHNDWZZXJKCULPJUQJOOODSJNPOWIMYHZXWGMGEFKJSCYSXQHHBMLTFGMSPGWYUZGQGMPVBYJEBJCXAQRXXWUIMBDJCCDNWIECWNQCTHSFJHTFKPNXAVESKBSFNUMBYCMXHTGZWCXSUEJDKEIUWRYWJKXRBROGFJRNZIYBJFLIZBMBDIDWYGOHZCLJWLKLFPNCBNSHXBKLLWUBWBISLVEFHUGVYJHQMXPSFJWAGBBFWHJNQXOYMHIAPWDHQVNEZGZIXOAYSJWYTLB', 243]


In [88]:
x = 30
print(len(s_arr[x][0]))
print(obj.characterReplacement(s_arr[x][0],s_arr[x][1]))
print(obj_3.characterReplacement(s_arr[x][0],s_arr[x][1]))

824
698
698


In [58]:
s = 'ABABBBA'
print(set(s))

{'B', 'A'}


In [34]:
def isValidString(s, length, k):
    l = 0

    boo = False

    # Buidling freduqency dictionary
    freq_dict = {}
    for i in range(length):
        if s[i] in freq_dict:
            freq_dict[s[i]] += 1
        else:
            freq_dict[s[i]] = 1
    
    for r in range((length-1), len(s)):

        # Check if this window is true
        max_freq = max(freq_dict.values())
        n_other = length - max_freq
        if n_other <= k:
            boo = True

        l += 1

        if r != (len(s)-1):
            # Shift window along
            freq_dict[s[l]] -= 1
            if s[r+1] in freq_dict:
                freq_dict[s[r+1]] += 1
            else:
                freq_dict[s[r+1]] = 1

    return boo

print(isValidString("ABAB", 4, 1))


False
